

---


**MÁSTER UNIVERSITARIO EN DATA ANALYTICS FOR BUSINESS** | **UPF Barcelona School of Management** | **Curso 2023-2024**

**Trabajo Final de Máster**

**Título del proyecto**: Análisis exploratorio de accidentes de tráfico en Barcelona

**Script**: Limpieza de datos

**Autores**: Joel Bullich Esquerra y Pablo García Prado

**Mentora**: Alexandra Abós Ortega


---




## Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import os
import random
from tabulate import tabulate

from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_columns', None)

Mounted at /content/drive


## Tratamiento de los *datasets*

### Anexionar los *datasets* del mismo tipo

La totalidad de nuestro conjunto de datos está almacenado en seis carpetas distintas de *Drive*, donde en cada una de ellas hay siete archivos .csv con información correspondiente a accidentes de tráfico. Los datos están almacenados en carpetas distintas porque en cada una se tratan variables distintas de cada accidente.

Para empezar, queremos pasar de tener 42 archivos a tener solamente 6 y así facilitar el tratamiento y limpieza. Entonces, vamos a anexar las observaciones de los que comparten exactamente las mismas variables.

In [ ]:
# Cambia al directorio que contiene las carpetas
os.chdir('/content/drive/MyDrive/TFM/raw_data')

# Lista para almacenar los DataFrames resultantes
dataframes = []

# Iterar sobre cada carpeta
for folder in os.listdir():
    if os.path.isdir(folder):
        # Crear una lista para almacenar los DataFrames de los archivos CSV de esta carpeta
        dfs = []
        # Iterar sobre los archivos CSV en la carpeta actual
        for file in os.listdir(folder):
            if file.endswith('.csv'):
                # Leer el archivo CSV y agregarlo a la lista
                df = pd.read_csv(os.path.join(folder, file))
                dfs.append(df)
        # Combinar los DataFrames en uno solo y agregarlo a la lista de DataFrames
        combined_df = pd.concat(dfs, ignore_index=True)
        dataframes.append(combined_df)

# Guardar cada DataFrame de la lista dataframes con nombres df_1, df_2, etc.
for i, df in enumerate(dataframes, start=1):
    globals()[f"df_{i}"] = df

Ahora tenemos seis *dataframes* distintos. El objetivo será juntar todos estos datasets, una vez limpios, a un único dataset.

In [ ]:
path = "/content/drive/MyDrive/TFM/raw_data/1/download.csv"
df1 = pd.read_csv(path)

path = "/content/drive/MyDrive/TFM/raw_data/1/download-2.csv"
df2 = pd.read_csv(path)

path = "/content/drive/MyDrive/TFM/raw_data/1/download-3.csv"
df3 = pd.read_csv(path)

path = "/content/drive/MyDrive/TFM/raw_data/1/download-4.csv"
df4 = pd.read_csv(path)

path = "/content/drive/MyDrive/TFM/raw_data/1/download-5.csv"
df5 = pd.read_csv(path)

path = "/content/drive/MyDrive/TFM/raw_data/1/download-6.csv"
df6 = pd.read_csv(path)

path = "/content/drive/MyDrive/TFM/raw_data/1/download-7.csv"
df7 = pd.read_csv(path)

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)
print(df7.shape)

(10510, 20)
(11091, 20)
(10835, 20)
(11025, 20)
(7595, 20)
(8979, 20)
(9320, 20)


In [ ]:
df_6.shape

(79647, 46)

### Análisis de los datasets para eliminar columnas repetidas

Con el objetivo de juntar los 7 datasets en un único dataset, en este apartado analizaremos cada uno de los datasets para ver si existen columnas duplicadas que podemos eliminar. Para poder ver el formato de los valores de las columnas de cada *dataframe*, hemos creado una función que muestra cinco valores aleatorios de cada una de las variables (excluyendo los valores nulos). Esta, nos va a permitir ver, más gráficamente que utilizando *.unique()*, cuáles podrían ser las columnas que contienen el mismo tipo de información (columnas duplicadas) más allá de su nombre.

In [ ]:
# Definimos la función
def mostrar_valores_unicos_aleatorios_tabla(df):
    tabla_valores_unicos = [] #creamos una lista que servirá para luego montarla en formato tabular

    for columna in df.columns: #recorremos cada columa de nuestro df
        valores_unicos = df[columna].dropna().unique()  # Para cada columna, eliminamos los NaN y obtemos los valores únicos
        if len(valores_unicos) > 5: #si tenemos, por lo menos, 5 valores únicos...
            valores_unicos_aleatorios = random.sample(list(valores_unicos.astype(str)), 5)  # Convertimos a cadena y escoger 5 valores aleatorios únicos
        else: #si hay más de cinco, lo convertimos a tipo string
            valores_unicos_aleatorios = valores_unicos.astype(str)
        # Truncamos las cadenas de caracteres a una longitud fija de 20 (max) para evitar errores en la tabulación
        valores_unicos_aleatorios = [valor[:20] for valor in valores_unicos_aleatorios]
        tabla_valores_unicos.append([columna] + valores_unicos_aleatorios) #añadimos a nuestra lista el nombre de las columnas junto a los 5 valores aleatorios

    # Imprimimos la tabla utilizando tabulate y escogemos los títulos de los encabezados
    print(tabulate(tabla_valores_unicos, headers=["Columna", "Valor 1", "Valor 2", "Valor 3", "Valor 4", "Valor 5"]))

#### *Dataframe* tipo 1

In [ ]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69355 entries, 0 to 69354
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Numero_expedient            69355 non-null  object 
 1   Codi_districte              69355 non-null  int64  
 2   Nom_districte               69355 non-null  object 
 3   Codi_barri                  69355 non-null  int64  
 4   Nom_barri                   69355 non-null  object 
 5   Codi_carrer                 69355 non-null  int64  
 6   Nom_carrer                  69354 non-null  object 
 7   Num_postal                  69339 non-null  object 
 8   Descripcio_dia_setmana      69355 non-null  object 
 9   NK_Any                      69355 non-null  int64  
 10  Mes_any                     69355 non-null  int64  
 11  Nom_mes                     69355 non-null  object 
 12  Dia_mes                     69355 non-null  int64  
 13  Hora_dia                    693

In [ ]:
mostrar_valores_unicos_aleatorios_tabla(df_1)

Columna                     Valor 1               Valor 2               Valor 3               Valor 4               Valor 5
--------------------------  --------------------  --------------------  --------------------  --------------------  --------------------
Numero_expedient            2019S009387           2020S005140           2022S003736           2019S007153           2022S005029
Codi_districte              5                     3                     8                     4                     1
Nom_districte               Les Corts             Sant Martí            Desconegut            Sants-Montjuïc        Sant Andreu
Codi_barri                  53                    61                    14                    9                     10
Nom_barri                   la Marina de Port     la Vila Olímpica del  Vallbona              el Guinardó           el Coll
Codi_carrer                 331507                120104                134007                367402                337106

A simple vista, ya encontramos que nuestro dataset contiene columnas que parecen dar el mismo tipo de información. En este caso, así como en los siguientes, lo que haremos será limitar el numero de columnas de manera que las que contienen la misma información se junten y se eliminen posteriormente.

En este dataset, en referencia a las coordenadas, hemos observado que existen diferentes sistemas de referencia (por ejemplo, el ED50) y que, además, pueden ser de diferentes tipos (por ejemplo, geográficas, UTM...). Sea el método que sea empleado, todas deberían ser equivalentes, ya que contienen el mismo tipo de información. (Fuente: https://www.ign.es/web/ca/calculadora-geodesica)

La idea, en este dataset, será comprobar si podemos rellenar la información de longitud y latitud con estos datos. De no ser, así decidiremos eliminar las columnas.

In [ ]:
filas_filtradas_Longitud = df_1.loc[df_1['Longitud'].isnull()&df_1['Coordenada_UTM_X_ED50'].notnull(),['Longitud','Coordenada_UTM_X_ED50']]
print(filas_filtradas_Longitud)

#hacemos lo mismo entre Latitud y la Coordenada_UTM_Y
filas_filtradas_Latitud = df_1.loc[df_1['Latitud'].isnull()&df_1['Coordenada_UTM_Y_ED50'].notnull(),['Latitud','Coordenada_UTM_Y_ED50']]
print(filas_filtradas_Latitud)

       Longitud  Coordenada_UTM_X_ED50
52129       NaN                   -1.0
52130       NaN                   -1.0
52131       NaN                   -1.0
52132       NaN                   -1.0
52133       NaN                   -1.0
52134       NaN                   -1.0
52135       NaN                   -1.0
53298       NaN                   -1.0
53299       NaN                   -1.0
54395       NaN                   -1.0
54396       NaN                   -1.0
54397       NaN                   -1.0
54398       NaN                   -1.0
55521       NaN                   -1.0
55522       NaN                   -1.0
55523       NaN                   -1.0
55524       NaN                   -1.0
55525       NaN                   -1.0
55526       NaN                   -1.0
56660       NaN                   -1.0
57819       NaN                   -1.0
57820       NaN                   -1.0
57821       NaN                   -1.0
57822       NaN                   -1.0
58944       NaN          

Debido a que no podemos saber estos valores (ya que -1.0 no es un valor válido), decidimos eliminarlas de nuestro dataset.

De la misma manera, eliminaremos Nom_mes, ya que incluye la misma información que Mes_any. También renombraremos la columna NK_Any a Any, para tenerlo parecido a los demás datasets.

In [ ]:
# Eliminar las columnas especificadas
df_1.drop(columns=['Coordenada_UTM_X_ED50', 'Coordenada_UTM_Y_ED50','Nom_mes'], inplace=True)
df_1.rename(columns={'NK_Any': 'Any'}, inplace=True)

In [ ]:
df_1.isna().sum()

Numero_expedient                  0
Codi_districte                    0
Nom_districte                     0
Codi_barri                        0
Nom_barri                         0
Codi_carrer                       0
Nom_carrer                        1
Num_postal                       16
Descripcio_dia_setmana            0
Any                               0
Mes_any                           0
Dia_mes                           0
Hora_dia                          0
Descripcio_torn                   0
Descripcio_causa_conductor    18299
Longitud                         30
Latitud                          30
Descripcio_causa_mediata      51056
dtype: int64

#### *Dataframe* tipo 2

In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65271 entries, 0 to 65270
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           54720 non-null  object 
 1   Codi_districte             65271 non-null  int64  
 2   Nom_districte              65271 non-null  object 
 3   Codi_barri                 65271 non-null  int64  
 4   Nom_barri                  65271 non-null  object 
 5   Codi_carrer                65271 non-null  int64  
 6   Nom_carrer                 65269 non-null  object 
 7   Num_postal_caption         49693 non-null  object 
 8   Descripcio_dia_setmana     54720 non-null  object 
 9   Dia_setmana                49377 non-null  object 
 10  Descripcio_tipus_dia       38826 non-null  object 
 11  Any                        32261 non-null  float64
 12  Mes_any                    54720 non-null  float64
 13  Nom_mes                    65271 non-null  obj

In [ ]:
mostrar_valores_unicos_aleatorios_tabla(df_2)

Columna                    Valor 1               Valor 2               Valor 3               Valor 4               Valor 5
-------------------------  --------------------  --------------------  --------------------  --------------------  --------------------
Numero_expedient           2016S008049           2020S002807           2017S004774           2020S002633           2022S003000
Codi_districte             5                     1                     6                     9                     8
Nom_districte              Sarrià-Sant Gervasi   Gràcia                Nou Barris            Horta-Guinardó        Sant Martí
Codi_barri                 12                    32                    60                    54                    33
Nom_barri                  la Sagrera            el Camp de l'Arpa de  el Parc i la Llacuna  la Salut              la Prosperitat
Codi_carrer                78008                 120909                701564                134608                193006
N

Siguiendo el caso anterior, y con el objetivo de seguir eliminando duplicidades, lo que haremos es escoger aquellas columnas que contengan la misma información y complementarlas para, finalmente, eliminar una de las dos duplicadas. En este caso, se realizarán los siguientes cambios:

- 'Numero_expedient' = 'Codi_expedient'. Ambos refieren al número identificativo de cada uno de los expedientes

- Tipus_dia = 'Descripcio_tipus_dia'. Indica si el día de la semana en cuestión es laborable o no lo es.

- 'Mes_any' = 'Nom_mes'. Indica el mes del año

- Descripcio_dia_setmana' = 'ID_Dia_setmana' y 'Dia_setmana'. Se refiere al nombre del dia de la semana en que ocurrió el incidente

- 'Any' = 'NK Any'. Año en el que ocurrió el accidente.

- 'Dia_mes' = 'Dia_de_mes'. Dia del mes en el que ocurrió el accidente.

- 'Hora_dia' = 'Hora_de_dia'. Hora del día en el que ocurrió el accidente.

- 'Descripcio_torn' = 'Torn'. Momento del día en que se notificó y detalló el incidente (noche, día, tarde...)

- 'Descripcio_tipus_accident' = 'Tipus_accident'. Descripción del accidente

- 'Longitud' = 'Longitud_WGS84'

- 'Latitud' = 'Latitud_WGS84'

- 'Num_postal' ='Num_postal '


Una vez hechas estas combinaciones, se eliminarán las columnas extras (las que se encuentrán después del '=') y algunas otras que no aportan información.

In [ ]:
# Fusionar las columnas que son correspondientes
df_2['Numero_expedient'] = df_2['Numero_expedient'].combine_first(df_2['Codi_expedient'])

df_2['Tipus_dia'] = df_2['Tipus_dia'].combine_first(df_2['Descripcio_tipus_dia'])

df_2['Mes_any'] = df_2['Mes_any'].combine_first(df_2['Nom_mes'])

df_2['Descripcio_dia_setmana'] = df_2['Descripcio_dia_setmana'].combine_first(df_2['ID_Dia_setmana']).combine_first(df_2['Dia_setmana'])

df_2['Descripcio_tipus_dia'] = df_2['Descripcio_tipus_dia'].combine_first(df_2['Tipus_dia'])

df_2['Any'] = df_2['Any'].combine_first(df_2['NK_Any'])

df_2['Dia_mes'] = df_2['Dia_mes'].combine_first(df_2['Dia_de_mes'])

df_2['Hora_dia'] = df_2['Hora_dia'].combine_first(df_2['Hora_de_dia'])

df_2['Descripcio_torn'] = df_2['Descripcio_torn'].combine_first(df_2['Torn'])

df_2['Descripcio_tipus_accident'] = df_2['Descripcio_tipus_accident'].combine_first(df_2['Tipus_accident'])

df_2['Longitud'] = df_2['Longitud'].combine_first(df_2['Longitud_WGS84'])

df_2['Latitud'] = df_2['Latitud'].combine_first(df_2['Latitud_WGS84'])

df_2['Num_postal'] = df_2['Num_postal '].combine_first(df_2['Num_postal_caption'])

# Eliminar las columnas sobrantes
df_2.drop(columns=['Codi_expedient', 'Num_postal ','Num_postal_caption',
                   'Coordenada_UTM_X_ED50', 'Coordenada_UTM_Y_ED50', 'Dia_setmana',
                   'ID_Dia_setmana', 'Tipus_dia', 'NK_Any', 'Nom_mes', 'Dia_de_mes', 'Hora_de_dia',
                   'Torn', 'Tipus_accident', 'Longitud_WGS84', 'Latitud_WGS84',
                   'Coordenada_UTM_X', 'Coordenada_UTM_Y', 'Descripcio_tipus_dia','Mes', 'Num_postal'], inplace=True)

In [ ]:
# Diccionario para mapear nombres de meses a números
meses_dict = {
    'Gener': 1,
    'Febrer': 2,
    'Març': 3,
    'Abril': 4,
    'Maig': 5,
    'Juny': 6,
    'Juliol': 7,
    'Agost': 8,
    'Setembre': 9,
    'Octubre': 10,
    'Novembre': 11,
    'Desembre': 12
}

# Convertir los nombres de los meses a números usando el diccionario
df_2['Mes_any'] = df_2['Mes_any'].replace(meses_dict)

# Convertir columnas a tipos adecuados
df_2['Any'] = df_2['Any'].astype(int)  # Convertir 'Any' a tipo entero
df_2['Mes_any'] = df_2['Mes_any'].astype(int)
df_2['Dia_mes'] = df_2['Dia_mes'].astype(int)  # Convertir 'Dia_mes' a tipo entero
df_2['Hora_dia'] = df_2['Hora_dia'].astype(int)  # Convertir 'Hora_dia' a tipo entero

# Eliminar ';' si está presente en la columna 'Longitud'
df_2['Longitud'] = df_2['Longitud'].astype(str).str.replace(';', '')

# Limpiar los valores de 'Longitud' para eliminar los caracteres no numéricos
df_2.loc[df_2['Longitud'] == '2.1722741.', 'Longitud'] = '2.1722741'

df_2['Longitud'] = df_2['Longitud'].astype(float)
df_2['Latitud'] = df_2['Latitud'].astype(float)

In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65271 entries, 0 to 65270
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           65271 non-null  object 
 1   Codi_districte             65271 non-null  int64  
 2   Nom_districte              65271 non-null  object 
 3   Codi_barri                 65271 non-null  int64  
 4   Nom_barri                  65271 non-null  object 
 5   Codi_carrer                65271 non-null  int64  
 6   Nom_carrer                 65269 non-null  object 
 7   Descripcio_dia_setmana     65271 non-null  object 
 8   Any                        65271 non-null  int64  
 9   Mes_any                    65271 non-null  int64  
 10  Dia_mes                    65271 non-null  int64  
 11  Hora_dia                   65271 non-null  int64  
 12  Descripcio_torn            65271 non-null  object 
 13  Descripcio_tipus_accident  65271 non-null  obj

In [ ]:
df_2.isna().sum()

Numero_expedient             0
Codi_districte               0
Nom_districte                0
Codi_barri                   0
Nom_barri                    0
Codi_carrer                  0
Nom_carrer                   2
Descripcio_dia_setmana       0
Any                          0
Mes_any                      0
Dia_mes                      0
Hora_dia                     0
Descripcio_torn              0
Descripcio_tipus_accident    0
Longitud                     2
Latitud                      2
dtype: int64

#### *Dataframe* tipo 3

In [ ]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62368 entries, 0 to 62367
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           62368 non-null  object 
 1   Codi_districte             62368 non-null  int64  
 2   Nom_districte              62368 non-null  object 
 3   Codi_barri                 62368 non-null  object 
 4   Nom_barri                  62368 non-null  object 
 5   Codi_carrer                62368 non-null  int64  
 6   Nom_carrer                 62366 non-null  object 
 7   Num_postal                 20474 non-null  object 
 8   Descripcio_dia_setmana     62368 non-null  object 
 9   Dia_setmana                46710 non-null  object 
 10  Descripcio_tipus_dia       46710 non-null  object 
 11  Any                        30415 non-null  float64
 12  Mes_any                    62368 non-null  int64  
 13  Nom_mes                    62368 non-null  obj

In [ ]:
mostrar_valores_unicos_aleatorios_tabla(df_3)

Columna                    Valor 1               Valor 2               Valor 3               Valor 4               Valor 5
-------------------------  --------------------  --------------------  --------------------  --------------------  --------------------
Numero_expedient           2019S010013           2020S002400           2022S001263           2018S005376           2022S000824
Codi_districte             10                    7                     -1                    4                     5
Nom_districte              Ciutat Vella          Sant Andreu           Les Corts             Desconegut            Nou Barris
Codi_barri                 12                    72                    24-2-7                91-9-63               81-8-54
Nom_barri                  la Barceloneta        la Bordeta            la Teixonera          Vilapicina i la Torr  Canyelles
Codi_carrer                104505                209004                240300                194100                352653
N

Seguimos con el mismo proceso que en los dfs anteriores. Haremos los siguientes cambios:

- 'Any' = 'NK_Any'

- 'Longitud' = 'Longitud_WGS84'

- 'Latitud' = 'Latitud_WGS84'

- 'Num_postal' = 'Num_postal ' y 'Num_postal_caption'

El motivo de juntar la longitud y latitud con su respectivas escalas de 'WGS84'es debido a que contienen exactamente la misma información. Simplemente, lo único que cambia es la notación del nombre de la variable. Sin embargo, en ambas el sistema de referencia utilizado es el de *World Geodetic System 1984*, por lo que se incluye el mismo tipo de información. (Fuente: https://www.ayuware.es/blog/coordenadas-wgs84/)

In [ ]:
# Fusionar las columnas que son correspondientes
df_3['Any'] = df_3['Any'].combine_first(df_3['NK_Any'])

df_3['Longitud'] = df_3['Longitud'].combine_first(df_3['Longitud_WGS84'])

df_3['Latitud'] = df_3['Latitud'].combine_first(df_3['Latitud_WGS84'])

df_3['Num_postal']=df_3['Num_postal'].combine_first(df_3['Num_postal ']).combine_first(df_3['Num_postal_caption'])

# Eliminar las columnas sobrantes
df_3.drop(columns=['Num_postal ', 'Num_postal_caption','Nom_mes','Descripcio_tipus_dia',
                   'Coordenada_UTM_X_ED50', 'Coordenada_UTM_Y_ED50', 'Dia_setmana',
                   'Longitud_WGS84', 'Latitud_WGS84', 'NK_Any',
                   'Coordenada_UTM_X', 'Coordenada_UTM_Y'], inplace=True)

df_3['Any'] = df_3['Any'].astype(int)
# df_3['Codi_barri'] = df_3['Codi_barri'].astype(int)

In [ ]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62368 entries, 0 to 62367
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           62368 non-null  object 
 1   Codi_districte             62368 non-null  int64  
 2   Nom_districte              62368 non-null  object 
 3   Codi_barri                 62368 non-null  object 
 4   Nom_barri                  62368 non-null  object 
 5   Codi_carrer                62368 non-null  int64  
 6   Nom_carrer                 62366 non-null  object 
 7   Num_postal                 57565 non-null  object 
 8   Descripcio_dia_setmana     62368 non-null  object 
 9   Any                        62368 non-null  int64  
 10  Mes_any                    62368 non-null  int64  
 11  Dia_mes                    62368 non-null  int64  
 12  Hora_dia                   62368 non-null  int64  
 13  Descripcio_torn            62368 non-null  obj

In [ ]:
df_3.isna().sum()

Numero_expedient                0
Codi_districte                  0
Nom_districte                   0
Codi_barri                      0
Nom_barri                       0
Codi_carrer                     0
Nom_carrer                      2
Num_postal                   4803
Descripcio_dia_setmana          0
Any                             0
Mes_any                         0
Dia_mes                         0
Hora_dia                        0
Descripcio_torn                 0
Descripcio_causa_vianant        0
Numero_morts                    0
Numero_lesionats_lleus          0
Numero_lesionats_greus          0
Numero_victimes                 0
Numero_vehicles_implicats       0
Longitud                       24
Latitud                        24
dtype: int64

#### *Dataframe* tipo 4

In [ ]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118274 entries, 0 to 118273
Data columns (total 37 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Codi_expedient            58283 non-null   object 
 1   Codi_districte            118274 non-null  int64  
 2   Nom_districte             118274 non-null  object 
 3   Codi_barri                118274 non-null  int64  
 4   Nom_barri                 118274 non-null  object 
 5   Codi_carrer               118274 non-null  int64  
 6   Nom_carrer                118217 non-null  object 
 7   Num_postal                65427 non-null   object 
 8   Descripcio_dia_setmana    118274 non-null  object 
 9   Dia_setmana               89020 non-null   object 
 10  Descripcio_tipus_dia      89020 non-null   object 
 11  Any                       58284 non-null   float64
 12  Mes_any                   118274 non-null  int64  
 13  Nom_mes                   118274 non-null  o

In [ ]:
mostrar_valores_unicos_aleatorios_tabla(df_4)

Columna                   Valor 1               Valor 2               Valor 3               Valor 4              Valor 5
------------------------  --------------------  --------------------  --------------------  -------------------  --------------------
Codi_expedient            2017S004488           2016S000453           2016S001197           2017S010095          2016S005466
Codi_districte            6                     7                     2                     5                    9
Nom_districte             Horta-Guinardó        Sarrià-Sant Gervasi   Gràcia                Sants-Montjuïc       Nou Barris
Codi_barri                6                     59                    37                    44                   10
Nom_barri                 Vallbona              la Sagrera            Ciutat Meridiana      Sant Andreu          el Poble-sec
Codi_carrer               63850                 277208                102805                311703               700060
Nom_carrer         

Aquí realizaremos los siguientes cambios:

- 'Codi_expedient' = 'Numero_expedient'. Ambos se refieren al número identificatibvo único de cada uno de los expedientes abiertos sobre los accidentes

- 'Num_postal' = 'Num_postal '

- 'Any' = 'NK_Any'

- Descripcio_dia_setmana = Dia_setmana

- 'Coordenada_UTM_X' = 'Coordenada_UTM_X_ED50'

- 'Coordenada_UTM_Y' = 'Coordenada_UTM_Y_ED50'

- 'Longitud' = 'Longitud_WGS84'

- 'Latitud' = 'Latitud_WGS84'

In [ ]:
# Fusionamos Numero_expedient con Codi_expedient
df_4['Numero_expedient'] = df_4['Numero_expedient'].combine_first(df_4['Codi_expedient'])
#en este caso queremos que numero_expedient sea la primera columna del df (queremos igualarlo a los demás dfs)
# Una vez juntas, eliminamos la columna 'Codi_expedient'
df_4.drop(columns='Codi_expedient', inplace=True)

#Fusionamos Num_postal con Num_postal y eliminamos Num_postal
df_4['Num_postal'] = df_4['Num_postal'].combine_first(df_4['Num_postal '])
df_4.drop(columns='Num_postal ', inplace=True)

#Fusionamos Any con NK_Any y eliminamos NK_Any
df_4['Any']=df_4['Any'].combine_first(df_4['NK_Any'])
df_4.drop(columns='NK_Any',inplace=True)

#Fusionamos Descripcio_dia_setmana con Dia_setmana y eliminamos Descripcio_dia_setmana
df_4['Descripcio_dia_setmana']=df_4['Descripcio_dia_setmana'].combine_first(df_4['Dia_setmana'])
df_4.drop(columns='Dia_setmana',inplace=True)

#Fusionamos Coordenada_UTM_X con Coordenada_UTM_X_ED50 y eliminamos Coordenada_UTM_X_ED50
df_4['Coordenada_UTM_X']=df_4['Coordenada_UTM_X'].combine_first(df_4['Coordenada_UTM_X_ED50'])
df_4.drop(columns='Coordenada_UTM_X_ED50',inplace=True)

#Fusionamos Coordenada_UTM_Y con Coordenada_UTM_Y_ED50 y eliminamos Coordenada_UTM_Y_ED50
df_4['Coordenada_UTM_Y']=df_4['Coordenada_UTM_Y'].combine_first(df_4['Coordenada_UTM_Y_ED50'])
df_4.drop(columns='Coordenada_UTM_Y_ED50',inplace=True)

#Fusionamos Longitud con Longitud_WGS84 y eliminamos Longitud_WGS84
df_4[' Longitud']=df_4[' Longitud'].combine_first(df_4['Longitud_WGS84'])
df_4.drop(columns='Longitud_WGS84',inplace=True)

#Fusionamos Latitud con Latitud_WGS84 y eliminamos Latitud_WGS84
df_4[' Latitud']=df_4[' Latitud'].combine_first(df_4['Latitud_WGS84'])
df_4.drop(columns='Latitud_WGS84',inplace=True)

#Fusionamos Longitud con ' Longitud' y eliminamos ' Longitud'
df_4['Longitud']=df_4['Longitud'].combine_first(df_4[' Longitud'])
df_4.drop(columns=' Longitud',inplace=True)

#Fusionamos Latitud con ' Latitud' y eliminamos ' Latitud'
df_4['Latitud']=df_4['Latitud'].combine_first(df_4[' Latitud'])
df_4.drop(columns=' Latitud',inplace=True)

#Convertimos las variables al tipo correcto
df_4['Any'] = df_4['Any'].astype(int)

df_4['Latitud'] = df_4['Latitud'].astype(str)
df_4['Longitud'] = df_4['Longitud'].astype(str)

# Reemplazar los valores de la columna 'Latitud' que son simplemente un string vacío por -1
df_4.loc[df_4['Latitud'] == "'", 'Latitud'] = -1
df_4.loc[df_4['Longitud'] == "'", 'Longitud'] = -1

df_4['Longitud'] = df_4['Longitud'].astype(float)
df_4['Latitud'] = df_4['Latitud'].astype(float)

Tratamos de reorganizar el df para que Numero_expedient aparezca el primero. Igual que en los casos anteriores, hemos comprobado que las columnas 'Coordenada_UTM_X' y 'Coordenada_UTM_Y' no nos pueden ser útiles para los valores NaN que existen en las columnas de Longitud y Latitud, respectivamente. Por ende, decidimos eliminarlas. Además, eliminamos las siguientes columnas que no son necesarias ya que, aquellas que incluyen la misma información, no tienen nulos. Finalmente, también eliminamos, los valores NaN de Numero_expedient, ya que nos resultará imposible encontrar los valores.

In [ ]:
ultima_columna = df_4.columns[-1]

# Creamos una lista con los nombres de las columnas, excluyendo la última
nombres_columnas = list(df_4.columns[:-1])

# Insertamos el nombre de la última columna en la primera posición
nombres_columnas.insert(0, ultima_columna)

# Reordenamos las columnas según la nueva lista de nombres de columnas
df_4 = df_4[nombres_columnas]

#eliminamos las columnas

df_4.drop(columns=['Coordenada_UTM_X','Coordenada_UTM_Y'], inplace=True)

df_4.drop(columns=['Descripcio_tipus_dia','Nom_mes'],inplace=True)

df_4.dropna(subset=['Numero_expedient'], inplace=True)

# Convertir al tipo correcto
df_4.loc[df_4['Antiguitat_carnet'] == "Desconegut", 'Antiguitat_carnet'] = -1
df_4['Antiguitat_carnet'] = df_4['Antiguitat_carnet'].astype(int)

<ipython-input-26-88dc9aae46ce>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4.drop(columns=['Coordenada_UTM_X','Coordenada_UTM_Y'], inplace=True)
<ipython-input-26-88dc9aae46ce>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4.drop(columns=['Descripcio_tipus_dia','Nom_mes'],inplace=True)
<ipython-input-26-88dc9aae46ce>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4.dropna(subset=['Numero_expedient'], inplace=True

In [ ]:
df_4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118251 entries, 0 to 118273
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Numero_expedient          118251 non-null  object 
 1   Codi_districte            118251 non-null  int64  
 2   Nom_districte             118251 non-null  object 
 3   Codi_barri                118251 non-null  int64  
 4   Nom_barri                 118251 non-null  object 
 5   Codi_carrer               118251 non-null  int64  
 6   Nom_carrer                118195 non-null  object 
 7   Num_postal                90312 non-null   object 
 8   Descripcio_dia_setmana    118251 non-null  object 
 9   Any                       118251 non-null  int64  
 10  Mes_any                   118251 non-null  int64  
 11  Dia_mes                   118251 non-null  int64  
 12  Hora_dia                  118251 non-null  int64  
 13  Descripcio_causa_vianant  118251 non-null  object

In [ ]:
df_4.isna().sum()

Numero_expedient                0
Codi_districte                  0
Nom_districte                   0
Codi_barri                      0
Nom_barri                       0
Codi_carrer                     0
Nom_carrer                     56
Num_postal                  27939
Descripcio_dia_setmana          0
Any                             0
Mes_any                         0
Dia_mes                         0
Hora_dia                        0
Descripcio_causa_vianant        0
Descripcio_tipus_vehicle        0
Descripcio_model                4
Descripcio_marca                0
Descripcio_color                0
Descripcio_carnet               0
Antiguitat_carnet               0
Longitud                       66
Latitud                        66
Descripcio_torn             39359
dtype: int64

#### *Dataframe* tipo 5

In [ ]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62357 entries, 0 to 62356
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Numero_expedient          52014 non-null  object 
 1   Codi_districte            62357 non-null  int64  
 2   Nom_districte             62357 non-null  object 
 3   Codi_barri                62357 non-null  int64  
 4   Nom_barri                 62357 non-null  object 
 5   Codi_carrer               62357 non-null  int64  
 6   Nom_carrer                62356 non-null  object 
 7   Num_postal                10153 non-null  object 
 8   Descripcio_dia_setmana    52014 non-null  object 
 9   Dia_setmana               46715 non-null  object 
 10  Descripcio_tipus_dia      36372 non-null  object 
 11  Any                       20083 non-null  float64
 12  Mes_any                   52014 non-null  float64
 13  Nom_mes                   62357 non-null  object 
 14  Dia_me

In [ ]:
mostrar_valores_unicos_aleatorios_tabla(df_5)

Columna                   Valor 1               Valor 2               Valor 3               Valor 4               Valor 5
------------------------  --------------------  --------------------  --------------------  --------------------  --------------------
Numero_expedient          2016S000037           2016S008733           2019S002419           2020S002502           2019S003640
Codi_districte            1                     7                     -1                    4                     9
Nom_districte             Les Corts             Nou Barris            Sants-Montjuïc        Gràcia                Sarrià-Sant Gervasi
Codi_barri                68                    49                    8                     13                    2
Nom_barri                 el Fort Pienc         el Besòs i el Maresm  Sarrià                Pedralbes             el Raval
Codi_carrer               138201                365000                701357                201807                197700
Nom_car

En este caso, de las 39 columnas, detectamos que hay muchas que siguen dando información duplicada. Con el mismo objetivo que en los casos anteriores, deseamos unir las columnas que tratan la misma información. En concreto, igualaremos la siguientes columnas:

- 'Numero_expedient' = 'Número_d'expedient'. Ambos refieren al número identificativo de cada uno de los expedientes

- 'Num_postal' ='Num_postal '

- Descripcio_dia_setmana' = 'Descripció_dia_setmana'. Se refiere al nombre del dia de la semana en que ocurrió el incidente

- Descripcio_tipus_dia = 'Descripció_tipus_dia'

- 'Any' = 'NK Any','NK_Any'

- 'Mes_any' = 'Mes_de_any'

- 'Dia_mes' = 'Dia_de_mes'

- 'Hora_dia' = 'Hora_de_dia'

- 'Descripcio_torn' = 'Descripció_torn'

- 'Descripcio_causa_mediata' = 'Descripció_causa_mediata'

- 'Coordenada_UTM_X'= 'Coordenada_UTM_(X)', 'Coordenads_UTM_X_ED50

- 'Coordenada_UTM_Y' = 'Coordenada_UTM_(Y)','Coordenada_UTM_Y_ED50

- 'Longitud' = 'Longitud_WGS84'

- 'Latitud' = 'Latitud_WGS84'







In [ ]:
#Fusionamos Numero_expedient con Número_d'expedient y eliminamos Número_d'expedient
df_5['Numero_expedient'] = df_5['Numero_expedient'].combine_first(df_5["Número_d'expedient"])
df_5.drop(columns="Número_d'expedient", inplace=True)

#Fusionamos Num_postal con Num_postal  y eliminamos Num_postal
df_5['Num_postal']=df_5['Num_postal'].combine_first(df_5['Num_postal '])
df_5.drop(columns='Num_postal ',inplace=True)

#Fusionamos Descripcio_dia_setmana con Descripció_dia_setmana y eliminamos Descripció_dia_setmana
df_5['Descripcio_dia_setmana']=df_5['Descripcio_dia_setmana'].combine_first(df_5['Descripció_dia_setmana'])
df_5.drop(columns='Descripció_dia_setmana',inplace=True)

#Fusionamos Descripcio_tipus_dia con Descripció_tipus_dia y eliminamos Descripció_tipus_dia
df_5['Descripcio_tipus_dia']=df_5['Descripcio_tipus_dia'].combine_first(df_5['Descripció_tipus_dia'])
df_5.drop(columns='Descripció_tipus_dia',inplace=True)

#Fusionamos Any con NK Any y NK_Any y eliminamos NK Any y NK_Any
df_5['Any']=df_5['Any'].combine_first(df_5['NK Any']).combine_first(df_5['NK_Any'])
df_5.drop(columns=['NK Any','NK_Any'],inplace=True)

#Fusionamos Mes_any con Mes_de_any y eliminamos Mes_de_any
df_5['Mes_any']=df_5['Mes_any'].combine_first(df_5['Mes_de_any'])
df_5.drop(columns=['Mes_de_any'],inplace=True)

#Fusionamos Dia_mes con Dia_de_mes y eliminamos Dia_de_mes
df_5['Dia_mes']=df_5['Dia_mes'].combine_first(df_5['Dia_de_mes'])
df_5.drop(columns=['Dia_de_mes'],inplace=True)

#Fusionamos Hora_dia con Hora_de_dia y eliminamos Hora_de_dia
df_5['Hora_dia']=df_5['Hora_dia'].combine_first(df_5['Hora_de_dia'])
df_5.drop(columns=['Hora_de_dia'],inplace=True)

#Fusionamos Descripcio_torn con Descripció_torn y eliminamos Descripció_torn
df_5['Descripcio_torn']=df_5['Descripcio_torn'].combine_first(df_5['Descripció_torn'])
df_5.drop(columns=['Descripció_torn'],inplace=True)

#Fusionamos Descripcio_causa_mediata con Descripció_causa_mediata y eliminamos Descripció_causa_mediata
df_5['Descripcio_causa_mediata']=df_5['Descripcio_causa_mediata'].combine_first(df_5['Descripció_causa_mediata'])
df_5.drop(columns=['Descripció_causa_mediata'],inplace=True)

#Fusionamos Coordenada_UTM_X con Coordenada_UTM_(X) y Coordenada_UTM_X_ED50 y eliminamos Coordenada_UTM_(X) y Coordenada_UTM_X_ED50
df_5['Coordenada_UTM_X']=df_5['Coordenada_UTM_X'].combine_first(df_5['Coordenada_UTM_(X)']).combine_first(df_5['Coordenada_UTM_X_ED50'])
df_5.drop(columns=['Coordenada_UTM_(X)','Coordenada_UTM_X_ED50'],inplace=True)

#Fusionamos Coordenada_UTM_Y con Coordenada_UTM_(Y) y Coordenada_UTM_Y_ED50 y eliminamos Coordenada_UTM_(Y) y Coordenada_UTM_Y_ED50
df_5['Coordenada_UTM_Y']=df_5['Coordenada_UTM_Y'].combine_first(df_5['Coordenada_UTM_(Y)']).combine_first(df_5['Coordenada_UTM_Y_ED50'])
df_5.drop(columns=['Coordenada_UTM_(Y)','Coordenada_UTM_Y_ED50'],inplace=True)

#Fusionamos Longitud con Longitud_WGS84 y eliminamos Longitud_WGS84
df_5['Longitud']=df_5['Longitud'].combine_first(df_5['Longitud_WGS84'])
df_5.drop(columns=['Longitud_WGS84'],inplace=True)

#Fusionamos Latitud con Latitud_WGS84 y eliminamos Latitud_WGS84
df_5['Latitud']=df_5['Latitud'].combine_first(df_5['Latitud_WGS84'])
df_5.drop(columns=['Latitud_WGS84'],inplace=True)

# Convertir columnas a tipos adecuados
df_5['Any'] = df_5['Any'].astype(int)  # Convertir 'Any' a tipo entero
df_5['Mes_any'] = df_5['Mes_any'].astype(int)
df_5['Dia_mes'] = df_5['Dia_mes'].astype(int)  # Convertir 'Dia_mes' a tipo entero
df_5['Hora_dia'] = df_5['Hora_dia'].astype(int)  # Convertir 'Hora_dia' a tipo entero

Antes de seguir eliminando columnas duplicadas, intentaremos ver si podemos rellenar los NaNs con las existentes.

En principio, nos gustaría mantener las columnas de Longitud y Latitud, y eliminar las Coordenadas_UTM_X y las Coordenadas_UTM_Y. Sin embargo, debido a la presencia de NaNs en las columnas de Longitud y Latitud, miraremos primero si podemos quitar esos NaNs reemplazándolos con los valores de las coordenadas UTM



Como, en ambos casos, no podemos reemplazar ningún valor nulo, decidimos eliminar las columnas de Coordenada_UTM_X y de Coordenada_UTM_Y. Además, eliminamos las siguientes columnas que no son necesarias ya que, aquellas que incluyen la misma información, no tienen nulos.

In [ ]:
#decidimos eliminar las columnas
df_5.drop(columns=['Coordenada_UTM_X','Coordenada_UTM_Y', 'Dia_setmana','Descripcio_tipus_dia','Nom_mes'],inplace=True)

In [ ]:
df_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62357 entries, 0 to 62356
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Numero_expedient          62357 non-null  object 
 1   Codi_districte            62357 non-null  int64  
 2   Nom_districte             62357 non-null  object 
 3   Codi_barri                62357 non-null  int64  
 4   Nom_barri                 62357 non-null  object 
 5   Codi_carrer               62357 non-null  int64  
 6   Nom_carrer                62356 non-null  object 
 7   Num_postal                57557 non-null  object 
 8   Descripcio_dia_setmana    62357 non-null  object 
 9   Any                       62357 non-null  int64  
 10  Mes_any                   62357 non-null  int64  
 11  Dia_mes                   62357 non-null  int64  
 12  Hora_dia                  62357 non-null  int64  
 13  Descripcio_torn           62357 non-null  object 
 14  Descri

In [ ]:
#observamos valores nulos
df_5.isna().sum()

Numero_expedient               0
Codi_districte                 0
Nom_districte                  0
Codi_barri                     0
Nom_barri                      0
Codi_carrer                    0
Nom_carrer                     1
Num_postal                  4800
Descripcio_dia_setmana         0
Any                            0
Mes_any                        0
Dia_mes                        0
Hora_dia                       0
Descripcio_torn                0
Descripcio_causa_mediata       0
Longitud                      23
Latitud                       23
dtype: int64

#### *Dataframe* tipo 6

In [ ]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79647 entries, 0 to 79646
Data columns (total 46 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Numero_expedient                          45139 non-null  object 
 1   Codi_districte                            79647 non-null  int64  
 2   Nom_districte                             79647 non-null  object 
 3   Codi_barri                                79647 non-null  int64  
 4   Nom_barri                                 79647 non-null  object 
 5   Codi_carrer                               79647 non-null  int64  
 6   Nom_carrer                                74359 non-null  object 
 7   Num_postal                                28798 non-null  object 
 8   Descripcio_dia_setmana                    79647 non-null  object 
 9   Dia_setmana                               61728 non-null  object 
 10  Descripcio_tipus_dia              

In [ ]:
mostrar_valores_unicos_aleatorios_tabla(df_6)

Columna                                   Valor 1               Valor 2               Valor 3               Valor 4               Valor 5
----------------------------------------  --------------------  --------------------  --------------------  --------------------  --------------------
Numero_expedient                          2022S003626           2022S004435           2018S003474           2017S005802           2016S001976
Codi_districte                            -1                    6                     2                     9                     7
Nom_districte                             Eixample              Gràcia                Sarrià-Sant Gervasi   Sarriŕ-Sant Gervasi   Desconegut
Codi_barri                                73                    42                    -1                    65                    50
Nom_barri                                 Hostafrancs           la Marina de Port     Can Baró              Sant Gervasi - Galva  Vallvidrera, el Tibi
Codi_carrer 

En este caso, de las 46 columnas, detectamos que hay muchas que siguen dando información duplicada. Con el mismo objetivo que en los casos anteriores, deseamos unir las columnas que tratan la misma información. En concreto, igualaremos la siguientes columnas:

- 'Numero_expedient' = 'Numero_Expedient'. Ambos refieren al número identificativo de cada uno de los expedientes

- 'Num_postal' ='Num_postal '

- 'Any' = 'NK_ Any'

- 'Mes_any' = 'Mes_ any'

- 'Descripcio_sexe' = 'Descripció_sexe'

- 'Descripcio_tipus_persona' = 'Descripció_tipus_persona'

- 'Coordenada_UTM_X'= 'Coordenada_UTM_X_ED50'

- 'Coordenada_UTM_Y' = 'Coordenada_UTM_Y_ED50'

- 'Longitud' = 'Longitud_WGS84'

- 'Latitud' = 'Latitud_WGS84'

- 'Descripcio_Lloc_atropellament_vianant' = 'Descripcio_Lloc_atropellament_vianat'

- 'Descripcio_Motiu_desplaçament_vianant' = 'Descripcio_Motiu_desplaĂ§ament_vianant' , 'Descripcio_Motiu_desplaÃ§ament_vianant'

- 'Descripcio_Motiu_desplaçament_conductor' = 'Descripcio_Motiu_desplaĂ§ament_conductor' , 'Descripcio_Motiu_desplaÃ§ament_conductor'

In [ ]:
#Fusionamos Numero_expedient con Numero_Expedient y eliminamos Numero_Expedient
df_6['Numero_expedient'] = df_6['Numero_expedient'].combine_first(df_6["Numero_Expedient"])
df_6.drop(columns="Numero_Expedient", inplace=True)

#Fusionamos Num_postal con Num_postal  y eliminamos Num_postal
df_6['Num_postal'] = df_6['Num_postal'].combine_first(df_6["Num_postal "])
df_6.drop(columns="Num_postal ", inplace=True)

#Fusionamos Any con NK_ Any  y eliminamos NK_ Any
df_6['Any'] = df_6['Any'].combine_first(df_6["NK_ Any"])
df_6.drop(columns="NK_ Any", inplace=True)

#Fusionamos Mes_any con Mes_ any  y eliminamos Mes_ any
df_6['Mes_any'] = df_6['Mes_any'].combine_first(df_6["Mes_ any"])
df_6.drop(columns="Mes_ any", inplace=True)

#Fusionamos Descripcio_sexe con Descripció_sexe y eliminamos Descripció_sexe
df_6['Descripcio_sexe'] = df_6['Descripcio_sexe'].combine_first(df_6["Descripció_sexe"])
df_6.drop(columns="Descripció_sexe", inplace=True)

#Fusionamos Descripcio_tipus_persona con Descripció_tipus_persona y eliminamos Descripció_tipus_persona
df_6['Descripcio_tipus_persona'] = df_6['Descripcio_tipus_persona'].combine_first(df_6["Descripció_tipus_persona"])
df_6.drop(columns="Descripció_tipus_persona", inplace=True)

#Fusionamos Coordenada_UTM_X con Coordenada_UTM_X_ED50 y eliminamos Coordenada_UTM_X_ED50
df_6['Coordenada_UTM_X'] = df_6['Coordenada_UTM_X'].combine_first(df_6["Coordenada_UTM_X_ED50"])
df_6.drop(columns="Coordenada_UTM_X_ED50", inplace=True)

#Fusionamos Coordenada_UTM_Y con Coordenada_UTM_Y_ED50 y eliminamos Coordenada_UTM_Y_ED50
df_6['Coordenada_UTM_Y'] = df_6['Coordenada_UTM_Y'].combine_first(df_6["Coordenada_UTM_Y_ED50"])
df_6.drop(columns="Coordenada_UTM_Y_ED50", inplace=True)

#Fusionamos Longitud con Longitud_WGS84 y eliminamos Longitud_WGS84
df_6['Longitud'] = df_6['Longitud'].combine_first(df_6["Longitud_WGS84"])
df_6.drop(columns="Longitud_WGS84", inplace=True)

#Fusionamos Latitud con Latitud_WGS84 y eliminamos Latitud_WGS84
df_6['Latitud'] = df_6['Latitud'].combine_first(df_6["Latitud_WGS84"])
df_6.drop(columns="Latitud_WGS84", inplace=True)

#Fusionamos Descripcio_Lloc_atropellament_vianant con Descripcio_Lloc_atropellament_vianat y eliminamos Descripcio_Lloc_atropellament_vianat
df_6['Descripcio_Lloc_atropellament_vianant'] = df_6['Descripcio_Lloc_atropellament_vianant'].combine_first(df_6["Descripcio_Lloc_atropellament_vianat"])
df_6.drop(columns="Descripcio_Lloc_atropellament_vianat", inplace=True)

#Fusionamos Descripcio_Motiu_desplaçament_vianant con Descripcio_Motiu_desplaĂ§ament_vianant y Descripcio_Motiu_desplaÃ§ament_vianant y eliminamos Descripcio_Motiu_desplaĂ§ament_vianant y Descripcio_Motiu_desplaÃ§ament_vianant
df_6['Descripcio_Motiu_desplaçament_vianant']=df_6['Descripcio_Motiu_desplaçament_vianant'].combine_first(df_6['Descripcio_Motiu_desplaĂ§ament_vianant']).combine_first(df_6['Descripcio_Motiu_desplaÃ§ament_vianant'])
df_6.drop(columns=['Descripcio_Motiu_desplaĂ§ament_vianant','Descripcio_Motiu_desplaÃ§ament_vianant'],inplace=True)

#Fusionamos Descripcio_Motiu_desplaçament_conductor con Descripcio_Motiu_desplaĂ§ament_conductor y Descripcio_Motiu_desplaÃ§ament_vianant y eliminamos Descripcio_Motiu_desplaĂ§ament_conductor y Descripcio_Motiu_desplaÃ§ament_conductor
df_6['Descripcio_Motiu_desplaçament_conductor']=df_6['Descripcio_Motiu_desplaçament_conductor'].combine_first(df_6['Descripcio_Motiu_desplaĂ§ament_conductor']).combine_first(df_6['Descripcio_Motiu_desplaÃ§ament_conductor'])
df_6.drop(columns=['Descripcio_Motiu_desplaĂ§ament_conductor','Descripcio_Motiu_desplaÃ§ament_conductor'],inplace=True)

# Convertir columnas a tipos adecuados
df_6['Any'] = df_6['Any'].astype(int)  # Convertir 'Any' a tipo entero
df_6['Mes_any'] = df_6['Mes_any'].astype(int)

# Convertir al tipo correcto
df_6.loc[df_6['Edat'] == "Desconegut", 'Edat'] = -1
df_6['Edat'] = df_6['Edat'].astype(int)

df_6['Latitud'] = df_6['Latitud'].astype(str)
df_6['Longitud'] = df_6['Longitud'].astype(str)

df_6['Latitud'] = df_6['Latitud'].str.replace(',', '.')
df_6['Longitud'] = df_6['Longitud'].str.replace(',', '.')

# Reemplazar los valores de la columna 'Latitud' que son simplemente un string vacío por -1
df_6.loc[df_6['Latitud'] == "'", 'Latitud'] = -1
df_6.loc[df_6['Longitud'] == "'", 'Longitud'] = -1

df_6['Longitud'] = df_6['Longitud'].astype(float)
df_6['Latitud'] = df_6['Latitud'].astype(float)

Antes de seguir eliminando columnas duplicadas, intentaremos ver si podemos rellenar los NaNs con las existentes.

En principio, nos gustaría mantener las columnas de Longitud y Latitud, y eliminar las Coordenadas_UTM_X y las Coordenadas_UTM_Y. Sin embargo, debido a la presencia de NaNs en las columnas de Longitud y Latitud, miraremos primero si podemos quitar esos NaNs reemplazándolos con los valores de las coordenadas UTM


Como, en ambos casos, no podemos reemplazar ningún valor nulo, decidimos eliminar las columnas de Coordenada_UTM_X y de Coordenada_UTM_Y. Además, eliminamos las siguientes columnas que no son necesarias ya que, aquellas que incluyen la misma información, no tienen nulos.

In [ ]:
#decidimos eliminar las columnas
df_6.drop(columns=['Coordenada_UTM_X','Coordenada_UTM_Y', 'Dia_setmana','Descripcio_tipus_dia','Nom_mes'],inplace=True)

In [ ]:
df_6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79647 entries, 0 to 79646
Data columns (total 26 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Numero_expedient                         79647 non-null  object 
 1   Codi_districte                           79647 non-null  int64  
 2   Nom_districte                            79647 non-null  object 
 3   Codi_barri                               79647 non-null  int64  
 4   Nom_barri                                79647 non-null  object 
 5   Codi_carrer                              79647 non-null  int64  
 6   Nom_carrer                               74359 non-null  object 
 7   Descripcio_dia_setmana                   79647 non-null  object 
 8   Any                                      79647 non-null  int64  
 9   Mes_any                                  79647 non-null  int64  
 10  Dia_mes                                  79647

In [ ]:
df_6.isna().sum()

Numero_expedient                               0
Codi_districte                                 0
Nom_districte                                  0
Codi_barri                                     0
Nom_barri                                      0
Codi_carrer                                    0
Nom_carrer                                  5288
Descripcio_dia_setmana                         0
Any                                            0
Mes_any                                        0
Dia_mes                                        0
Descripcio_torn                                0
Hora_dia                                       0
Descripcio_causa_vianant                       0
Desc_Tipus_vehicle_implicat                    0
Descripcio_sexe                                0
Edat                                           0
Descripcio_tipus_persona                       0
Descripcio_situacio                        43573
Descripcio_victimitzacio                       0
Longitud            

In [ ]:
df_6[df_6['Codi_carrer']==32700]['Nom_carrer'].value_counts()

Nom_carrer
Balmes                                                675
Balmes                                                 85
Diagonal / Gràcia                                      12
Balmes / Còrsega                                        9
Aragó                                                   7
Aribau                                                  6
Balmes / València                                       6
Balmes / Provença                                       6
Balmes / Aragó                                          5
Balmes / Rosselló                                       5
Diagonal / Aribau                                       5
Balmes / Corts Catalanes                                5
Aragó / Catalunya                                       5
Balmes / Gràcia                                         4
Mallorca / Aribau                                       4
Balmes / Mallorca                                       4
Aragó / Balmes                                          4
Bal

### Convertir los *dataframes* a .csv

In [ ]:
dataframes = [df_1, df_2, df_3, df_4, df_5, df_6]

for df in dataframes:
    if 'Numero_expedient' in df.columns:
        df['Numero_expedient'] = df['Numero_expedient'].str.strip()

<ipython-input-42-c2498fd133ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Numero_expedient'] = df['Numero_expedient'].str.strip()


In [ ]:
for df in dataframes:
  df.sort_values(by='Numero_expedient', inplace=True)
  df.drop_duplicates(inplace=True)

print('Número de observaciones del dataframe 1:', df_1.shape)
print('Número de observaciones del dataframe 2:', df_2.shape)
print('Número de observaciones del dataframe 3:', df_3.shape)
print('Número de observaciones del dataframe 4:', df_4.shape)
print('Número de observaciones del dataframe 5:', df_5.shape)
print('Número de observaciones del dataframe 6:', df_6.shape)

Número de observaciones del dataframe 1: (69355, 18)
Número de observaciones del dataframe 2: (65271, 16)
Número de observaciones del dataframe 3: (62368, 22)
Número de observaciones del dataframe 4: (118251, 23)
Número de observaciones del dataframe 5: (62357, 17)
Número de observaciones del dataframe 6: (79647, 26)


In [ ]:
for i, df in enumerate(dataframes, start=1):
    if df['Numero_expedient'].isna().any():
        print(f"Se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df{i}.")
    else:
        print(f"No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df{i}.")

No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df1.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df2.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df3.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df4.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df5.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df6.


In [ ]:
df_1['Any'].value_counts()

Any
2017    11091
2019    11025
2018    10835
2016    10510
2022     9320
2021     8979
2020     7595
Name: count, dtype: int64

In [ ]:
df_2['Any'].value_counts()

Any
2017    11009
2016    10701
2018    10551
2019    10536
2022     8098
2021     7796
2020     6580
Name: count, dtype: int64

In [ ]:
df_3['Any'].value_counts()

Any
2017    10339
2016    10140
2019    10027
2018     9936
2022     7999
2021     7659
2020     6268
Name: count, dtype: int64

In [ ]:
df_4['Any'].value_counts()

Any
2017    19795
2016    19409
2018    19079
2019    19025
2022    14901
2021    14343
2020    11699
Name: count, dtype: int64

In [ ]:
df_5['Any'].value_counts()

Any
2017    10343
2016    10157
2019    10028
2018     9926
2022     7989
2021     7653
2020     6261
Name: count, dtype: int64

In [ ]:
df_6['Any'].value_counts()

Any
2020    13810
2017    12148
2016    12072
2018    11854
2019    11844
2022     9065
2021     8854
Name: count, dtype: int64

In [ ]:
# Directorio donde se almacenarán los archivos CSV limpios
output_dir = '/content/drive/MyDrive/TFM/clean_data'

# Crear el directorio si no existe
os.makedirs(output_dir, exist_ok=True)

# Guardar el DataFrame general como un archivo CSV
#df_1.to_csv(os.path.join(output_dir, "DatosCausaConductor.csv"), index=False)
#df_2.to_csv(os.path.join(output_dir, "DatosTipoAccidente.csv"), index=False)
#df_3.to_csv(os.path.join(output_dir, "DatosPersonasHeridas.csv"), index=False)
#df_4.to_csv(os.path.join(output_dir, "DatosTipoVehiculo.csv"), index=False)
#df_5.to_csv(os.path.join(output_dir, "DatosCausaMediata.csv"), index=False)
#df_6.to_csv(os.path.join(output_dir, "DatosPersona.csv"), index=False)

Ahora podemos encontrar los *dataframes* limpios en la ruta '/content/drive/MyDrive/TFM/clean_data'

## Juntar los seis *dataframes*

### Ordenar las columnas de los dfs

In [ ]:
# vemos las columnas de cada df
df1_columns = pd.DataFrame({'Columnas_df1': df_1.columns})
df2_columns = pd.DataFrame({'Columnas_df2': df_2.columns})
df3_columns = pd.DataFrame({'Columnas_df3': df_3.columns})
df4_columns = pd.DataFrame({'Columnas_df4': df_4.columns})
df5_columns = pd.DataFrame({'Columnas_df5': df_5.columns})
df6_columns = pd.DataFrame({'Columnas_df6': df_6.columns})

# Concatenamos horizontalmnente
df_columnas = pd.concat([df1_columns, df2_columns, df3_columns, df4_columns, df5_columns, df6_columns], axis=1)

df_columnas

,Columnas_df1,Columnas_df2,Columnas_df3,Columnas_df4,Columnas_df5,Columnas_df6
0,Numero_expedient,Numero_expedient,Numero_expedient,Numero_expedient,Numero_expedient,Numero_expedient
1,Codi_districte,Codi_districte,Codi_districte,Codi_districte,Codi_districte,Codi_districte
2,Nom_districte,Nom_districte,Nom_districte,Nom_districte,Nom_districte,Nom_districte
3,Codi_barri,Codi_barri,Codi_barri,Codi_barri,Codi_barri,Codi_barri
4,Nom_barri,Nom_barri,Nom_barri,Nom_barri,Nom_barri,Nom_barri
5,Codi_carrer,Codi_carrer,Codi_carrer,Codi_carrer,Codi_carrer,Codi_carrer
6,Nom_carrer,Nom_carrer,Nom_carrer,Nom_carrer,Nom_carrer,Nom_carrer
7,Num_postal,Descripcio_dia_setmana,Num_postal,Num_postal,Num_postal,Descripcio_dia_setmana
8,Descripcio_dia_setmana,Any,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Any
9,Any,Mes_any,Any,Any,Any,Mes_any


In [ ]:
#NUM_POSTAL
# Extraer la columna en el índice 17 y guardarla en una variable temporal
columna = df_2.pop(df_2.columns[16])
# Insertar la columna en el índice 7
df_2.insert(7, columna.name, columna)

# Extraer la columna en el índice 17 y guardarla en una variable temporal
columna = df_6.pop(df_6.columns[22])
# Insertar la columna en el índice 7
df_6.insert(7, columna.name, columna)

#DESCRIPCIO TORN
columna = df_6.pop(df_6.columns[12])
df_6.insert(13, columna.name,columna)

columna = df_4.pop(df_4.columns[22])
df_4.insert(13, columna.name,columna)

#LONGITUD Y LATITUD
columna = df_1.pop(df_1.columns[17])
df_1.insert(15, columna.name,columna)

columna = df_6.pop(df_6.columns[21])
df_6.insert(24, columna.name,columna)

columna = df_6.pop(df_6.columns[21])
df_6.insert(25,columna.name,columna)

columna = df_6.pop(df_6.columns[23])
df_6.insert(24,columna.name,columna)

IndexError: index 16 is out of bounds for axis 0 with size 16

In [ ]:
# vemos las columnas de cada df
df1_columns = pd.DataFrame({'Columnas_df1': df_1.columns})
df2_columns = pd.DataFrame({'Columnas_df2': df_2.columns})
df3_columns = pd.DataFrame({'Columnas_df3': df_3.columns})
df4_columns = pd.DataFrame({'Columnas_df4': df_4.columns})
df5_columns = pd.DataFrame({'Columnas_df5': df_5.columns})
df6_columns = pd.DataFrame({'Columnas_df6': df_6.columns})

# Concatenamos horizontalmnente
df_columnas = pd.concat([df1_columns, df2_columns, df3_columns, df4_columns, df5_columns, df6_columns], axis=1)

df_columnas

Ahora ya tenemos todas las columnas bien ordenadas en nuestro conjunto de datasets.

A continuacón, nos interesa gestionar los valores duplicados en el número de expediente, que será la clave por la cual vamos a juntar los datasets.

### Valores duplicados en número de expediente

Hay que tener en cuenta los duplicados de Numero de expediente en cada uno de los datasets. En este apartado, queremos ver si realmente existen estos valores duplicados.

Tomaremos como ejemplo los valores de numero de expediente repetidos en un df de ejemplo, el 4 en este caso.

In [ ]:
df_4['Numero_expedient'].nunique()

62262

In [ ]:
df_4.shape

(118251, 23)

A simple vista, vemos como hay algo que no cuadra. Nuestro dataframe tiene una longitud de 118251, y tenemos 62262 valores distintos de numero de expediente. Como ya se han eliminado los NaNs anteriormente, sabemos que existirán valores reptidos. A continuación, procedemos a explorar algunos ejemplos de valores repetidos

In [ ]:
#usamos la función 'duplicated'
duplicados_df4 = df_4[df_4.duplicated(subset='Numero_expedient', keep=False)]
duplicados_df4

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,Any,Mes_any,Dia_mes,Hora_dia,Descripcio_causa_vianant,Descripcio_tipus_vehicle,Descripcio_model,Descripcio_marca,Descripcio_color,Descripcio_carnet,Antiguitat_carnet,Longitud,Latitud,Descripcio_torn
6575,2016S000001,2,Eixample,7,la Dreta de l'Eixample,144601,Diagonal,0361B0361B,Divendres,2016,1,1,0,No és causa del vianant,Turismo,S40,VOLVO,Blau,B,53,2.163202,41.397223,NaN
6574,2016S000001,2,Eixample,7,la Dreta de l'Eixample,144601,Diagonal,0361B0361B,Divendres,2016,1,1,0,No és causa del vianant,Taxi,Altea,SEAT,Altres,BTP,10,2.163202,41.397223,NaN
15781,2016S000003,8,Nou Barris,53,la Trinitat Nova,209900,Meridiana,0595X0595X,Divendres,2016,1,1,5,No és causa del vianant,Motocicleta,SH 125,HONDA,Negre,B,4,2.187925,41.448446,NaN
15782,2016S000003,8,Nou Barris,53,la Trinitat Nova,209900,Meridiana,0595X0595X,Divendres,2016,1,1,5,No és causa del vianant,Turismo,307,PEUGEOT,Negre,B,11,2.187925,41.448446,NaN
5318,2016S000004,2,Eixample,8,l'Antiga Esquerra de l'Eixample,32700,Balmes,0119 0119,Divendres,2016,1,1,6,No és causa del vianant,Taxi,Altea,SEAT,Negre/Groc,B,8,2.158167,41.392530,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113213,2022S008012,6,Gràcia,31,la Vila de Gràcia,90502,Còrsega ...,0339 0339,Dimarts,2022,12,6,10,No és causa del vianant,Turisme,FIAT PANDA,FIAT,Blanc,B,7,2.160867,41.397829,Matí
107068,2022S008013,2,Eixample,8,l'Antiga Esquerra de l'Eixample,32700,Balmes ...,0083 0083,Dissabte,2022,12,10,1,No és causa del vianant,Motocicleta,SEAT MÓ ESCOOTER 125,SILENCE,Desconegut,B,9,2.160225,41.390994,Nit
107069,2022S008013,2,Eixample,8,l'Antiga Esquerra de l'Eixample,32700,Balmes ...,0083 0083,Dissabte,2022,12,10,1,No és causa del vianant,Taxi,Desconegut,Desconegut,Desconegut,Desconegut,-1,2.160225,41.390994,Nit
112980,2023S000004,6,Gràcia,28,Vallcarca i els Penitents,279600,República Argentina ...,0068 0068,Dissabte,2022,12,31,15,No és causa del vianant,Bicicleta,Desconegut,Desconegut,Negre,B,23,2.146288,41.409862,Tarda


#### Gestión y corrección de valores duplicados

Como hemos observado, tenemos valores duplicados en la columna 'Numero_expedient'. Como nos interesa mantener la información de las distintas observaciones - ya que cada una de las filas duplicadas da información diferente, por ejemplo información de los diferentes vehículos implicados en el accidete (uno por cada fila)- el objetivo será añadir columnas para almacenar la información de cada accidente en una sola fila, en lugar de en varias. Todo esto con el objetivo de poder juntar los seis dataframes en uno solo.

Observamos que hay muchos valores que tienen espacios después de los carácteres alfanuméricos. Eliminamos estos espacios, mediante la función "strip()"

In [ ]:
dataframes = [df_1, df_2, df_3, df_4, df_5, df_6]

for df in dataframes:
    if 'Numero_expedient' in df.columns:
        df['Numero_expedient'] = df['Numero_expedient'].str.strip()

In [ ]:
for df in dataframes:
  df.sort_values(by='Numero_expedient', inplace=True)
  df.drop_duplicates(inplace=True)

print('Número de observaciones del dataframe 1:', df_1.shape)
print('Número de observaciones del dataframe 2:', df_2.shape)
print('Número de observaciones del dataframe 3:', df_3.shape)
print('Número de observaciones del dataframe 4:', df_4.shape)
print('Número de observaciones del dataframe 5:', df_5.shape)
print('Número de observaciones del dataframe 6:', df_6.shape)

Número de observaciones del dataframe 1: (69355, 18)
Número de observaciones del dataframe 2: (65271, 16)
Número de observaciones del dataframe 3: (62368, 22)
Número de observaciones del dataframe 4: (118251, 23)
Número de observaciones del dataframe 5: (62357, 17)
Número de observaciones del dataframe 6: (79647, 26)


También comprobaremos que no haya ningún "NaN" que nos dificulte la conversión.

In [ ]:
for i, df in enumerate(dataframes, start=1):
    if df['Numero_expedient'].isna().any():
        print(f"Se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df{i}.")
    else:
        print(f"No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df{i}.")

No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df1.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df2.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df3.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df4.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df5.
No se encontraron NaN en la columna 'Numero_expedient' en el DataFrame df6.


##### Dataframe tipo 1

In [ ]:
df_1['Numero_expedient'].value_counts()

Numero_expedient
2018S005514    5
2020S005949    4
2021S001038    4
2022S007550    4
2020S005239    4
              ..
2018S000847    1
2018S000848    1
2018S000849    1
2018S000850    1
2023S000550    1
Name: count, Length: 62232, dtype: int64

In [ ]:
# Combinar las filas idénticas en una sola fila
df1 = df_1.groupby(['Numero_expedient', 'Codi_carrer']).agg({'Descripcio_torn': ', '.join}).reset_index()

In [ ]:
df1['Numero_expedient'].value_counts()

Numero_expedient
2016S000001    1
2020S001186    1
2020S001158    1
2020S001159    1
2020S001160    1
              ..
2018S000356    1
2018S000357    1
2018S000358    1
2018S000359    1
2023S000550    1
Name: count, Length: 62232, dtype: int64

##### Dataframe tipo 2

In [ ]:
df_2['Numero_expedient'].value_counts()

Numero_expedient
2016S008302    4
2019S004374    4
2020S003968    4
2018S003112    4
2019S004185    4
              ..
2018S000913    1
2018S000915    1
2018S000916    1
2018S000917    1
2023S000550    1
Name: count, Length: 62332, dtype: int64

In [ ]:
# Combinar las filas idénticas en una sola fila
df2 = df_2.groupby(['Numero_expedient', 'Codi_carrer']).agg({'Descripcio_tipus_accident': ', '.join}).reset_index()

df2.rename(columns={'Codi_carrer': 'Codi_carrer2'}, inplace=True)

In [ ]:
df2['Numero_expedient'].value_counts()

Numero_expedient
2016S000001    1
2020S001154    1
2020S001126    1
2020S001127    1
2020S001128    1
              ..
2018S000308    1
2018S000309    1
2018S000310    1
2018S000311    1
2023S000550    1
Name: count, Length: 62332, dtype: int64

##### Dataframe tipo 3

In [ ]:
df_3['Numero_expedient'].value_counts()

Numero_expedient
2018S000183    2
2020S000778    2
2020S004479    2
2018S003156    2
2022S002608    2
              ..
2018S000310    1
2018S000311    1
2018S000312    1
2018S000313    1
2023S000550    1
Name: count, Length: 62331, dtype: int64

In [ ]:
# Combinar las filas idénticas en una sola fila
df3 = df_3.groupby(['Numero_expedient', 'Codi_carrer', 'Numero_morts', 'Numero_lesionats_lleus', 'Numero_lesionats_greus',
     'Numero_victimes', 'Numero_vehicles_implicats']).agg({'Descripcio_causa_vianant': ', '.join}).reset_index()

df3.rename(columns={'Codi_carrer': 'Codi_carrer3'}, inplace=True)

In [ ]:
df3['Numero_expedient'].value_counts()

Numero_expedient
2016S000001    1
2020S001154    1
2020S001126    1
2020S001127    1
2020S001128    1
              ..
2018S000308    1
2018S000309    1
2018S000310    1
2018S000311    1
2023S000550    1
Name: count, Length: 62331, dtype: int64

##### Dataframe tipo 4

In [ ]:
df_4['Numero_expedient'].value_counts()

Numero_expedient
2019S008334    18
2019S001628    17
2019S003298    16
2021S002120    14
2017S000147    13
               ..
2021S002709     1
2021S002711     1
2021S002712     1
2017S000649     1
2023S000550     1
Name: count, Length: 62262, dtype: int64

In [ ]:
# Convertir las columnas con valores mixtos en cadenas
df_4[['Descripcio_causa_vianant', 'Descripcio_tipus_vehicle', 'Descripcio_model',
       'Descripcio_marca', 'Descripcio_color', 'Descripcio_carnet', 'Codi_carrer', 'Numero_expedient', 'Antiguitat_carnet']] = df_4[['Descripcio_causa_vianant', 'Descripcio_tipus_vehicle', 'Descripcio_model',
                                                                            'Descripcio_marca', 'Descripcio_color', 'Descripcio_carnet', 'Codi_carrer', 'Numero_expedient', 'Antiguitat_carnet']].astype(str)

# Combinar las filas idénticas en una sola fila
df4 = df_4.groupby(['Numero_expedient', 'Codi_carrer']).agg({
    'Descripcio_causa_vianant': ', '.join,
    'Descripcio_tipus_vehicle': ', '.join,
    'Descripcio_model': ', '.join,
    'Descripcio_marca': ', '.join,
    'Descripcio_color': ', '.join,
    'Descripcio_carnet': ', '.join,
    'Antiguitat_carnet': ', '.join
}).reset_index()

df4.rename(columns={'Codi_carrer': 'Codi_carrer4'}, inplace=True)

In [ ]:
df4['Numero_expedient'].value_counts()

Numero_expedient
2016S000001    1
2020S001161    1
2020S001133    1
2020S001134    1
2020S001135    1
              ..
2018S000317    1
2018S000318    1
2018S000319    1
2018S000320    1
2023S000550    1
Name: count, Length: 62262, dtype: int64

##### Dataframe tipo 5

In [ ]:
df_5['Numero_expedient'].value_counts()

Numero_expedient
2016S007882    2
2016S003497    2
2016S006066    2
2016S007767    2
2016S006144    2
              ..
2018S000325    1
2018S000326    1
2018S000327    1
2018S000328    1
2023S000550    1
Name: count, Length: 62332, dtype: int64

In [ ]:
# Combinar las filas idénticas en una sola fila
df5 = df_5.groupby(['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri', 'Nom_barri', 'Codi_carrer', 'Nom_carrer', #'Num_postal',
    'Descripcio_dia_setmana', 'Any', 'Mes_any', 'Dia_mes', 'Hora_dia', 'Descripcio_torn', 'Longitud', 'Latitud']).agg({'Descripcio_causa_mediata': ', '.join}).reset_index()

df5.rename(columns={'Codi_carrer': 'Codi_carrer5'}, inplace=True)

In [ ]:
df5['Numero_expedient'].value_counts()

Numero_expedient
2016S000001    1
2020S001138    1
2020S001110    1
2020S001111    1
2020S001112    1
              ..
2018S000300    1
2018S000301    1
2018S000302    1
2018S000303    1
2023S000550    1
Name: count, Length: 62308, dtype: int64

##### Dataframe tipo 6

In [ ]:
df_6['Numero_expedient'].value_counts()

Numero_expedient
2016S006967    21
2019S004830    21
2019S006444    12
2018S006493    12
2021S003925    12
               ..
2018S002959     1
2018S002961     1
2018S002962     1
2018S002963     1
2023S000550     1
Name: count, Length: 57686, dtype: int64

In [ ]:
# Convertir las columnas con valores mixtos en cadenas
df_6[['Descripcio_sexe', 'Edat', 'Descripcio_tipus_persona', 'Descripcio_situacio', 'Descripcio_victimitzacio',
      'Descripcio_Motiu_desplaçament_vianant', 'Descripcio_Motiu_desplaçament_conductor',
      'Descripcio_Lloc_atropellament_vianant']] = df_6[['Descripcio_sexe', 'Edat', 'Descripcio_tipus_persona',
                                                       'Descripcio_situacio', 'Descripcio_victimitzacio',
                                                       'Descripcio_Motiu_desplaçament_vianant',
                                                       'Descripcio_Motiu_desplaçament_conductor',
                                                       'Descripcio_Lloc_atropellament_vianant']].astype(str)

# Combinar las filas idénticas en una sola fila
df6 = df_6.groupby(['Numero_expedient', 'Codi_carrer']).agg({
    'Descripcio_sexe': ', '.join,
    'Edat': ', '.join,
    'Descripcio_tipus_persona': ', '.join,
    'Descripcio_situacio': ', '.join,
    'Descripcio_victimitzacio': ', '.join,
    'Descripcio_Motiu_desplaçament_vianant': ', '.join,
    'Descripcio_Motiu_desplaçament_conductor': ', '.join,
    'Descripcio_Lloc_atropellament_vianant': ', '.join
}).reset_index()

df6.rename(columns={'Codi_carrer': 'Codi_carrer6'}, inplace=True)


In [ ]:
df6['Numero_expedient'].value_counts()

Numero_expedient
2016S000001    1
2020S001207    1
2020S001237    1
2020S001238    1
2020S001239    1
              ..
2018S000498    1
2018S000499    1
2018S000500    1
2018S000501    1
2023S000550    1
Name: count, Length: 57686, dtype: int64

##### Merge

In [ ]:
dataframes = [df1, df2, df3, df4, df5, df6]

for df in dataframes:
  df.sort_values(by='Numero_expedient', inplace=True)
  df.drop_duplicates(inplace=True)

print('Número de observaciones del dataframe 1:', df1.shape)
print('Número de observaciones del dataframe 2:', df2.shape)
print('Número de observaciones del dataframe 3:', df3.shape)
print('Número de observaciones del dataframe 4:', df4.shape)
print('Número de observaciones del dataframe 5:', df5.shape)
print('Número de observaciones del dataframe 6:', df6.shape)

Número de observaciones del dataframe 1: (62232, 3)
Número de observaciones del dataframe 2: (62332, 3)
Número de observaciones del dataframe 3: (62331, 8)
Número de observaciones del dataframe 4: (62262, 9)
Número de observaciones del dataframe 5: (62308, 16)
Número de observaciones del dataframe 6: (57686, 10)


In [ ]:
df_general = pd.merge(df5, df3, on = ['Numero_expedient'], how = 'left')

print(df_general.columns)
print(df_general.shape)

df_general2 = pd.merge(df_general, df2, on = ['Numero_expedient'], how = 'left')

print(df_general2.columns)
print(df_general2.shape)

df_general3 = pd.merge(df_general2, df4, on = ['Numero_expedient'], how = 'left')

print(df_general3.columns)
print(df_general3.shape)

df_general4 = pd.merge(df_general3, df6, on = ['Numero_expedient'], how = 'left')

print(df_general4.columns)
print(df_general4.shape)

Index(['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Codi_carrer5', 'Nom_carrer', 'Descripcio_dia_setmana',
       'Any', 'Mes_any', 'Dia_mes', 'Hora_dia', 'Descripcio_torn', 'Longitud',
       'Latitud', 'Descripcio_causa_mediata', 'Codi_carrer3', 'Numero_morts',
       'Numero_lesionats_lleus', 'Numero_lesionats_greus', 'Numero_victimes',
       'Numero_vehicles_implicats', 'Descripcio_causa_vianant'],
      dtype='object')
(62308, 23)
Index(['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Codi_carrer5', 'Nom_carrer', 'Descripcio_dia_setmana',
       'Any', 'Mes_any', 'Dia_mes', 'Hora_dia', 'Descripcio_torn', 'Longitud',
       'Latitud', 'Descripcio_causa_mediata', 'Codi_carrer3', 'Numero_morts',
       'Numero_lesionats_lleus', 'Numero_lesionats_greus', 'Numero_victimes',
       'Numero_vehicles_implicats', 'Descripcio_causa_vianant', 'Codi_carrer2',
       'Descripcio_tipus_accident'],
      dtyp

In [ ]:
# Eliminar columnas
df_general4.drop(columns=['Codi_districte', 'Codi_barri',# 'Num_postal',
                          'Descripcio_causa_vianant_y'], inplace=True)

# Renombrar columna
df_general4.rename(columns={'Descripcio_causa_vianant_x': 'Descripcio_causa_vianant'}, inplace=True)

In [ ]:
df_general4.isna().sum()

Numero_expedient                              0
Nom_districte                                 0
Nom_barri                                     0
Codi_carrer5                                  0
Nom_carrer                                    0
Descripcio_dia_setmana                        0
Any                                           0
Mes_any                                       0
Dia_mes                                       0
Hora_dia                                      0
Descripcio_torn                               0
Longitud                                      0
Latitud                                       0
Descripcio_causa_mediata                      0
Codi_carrer3                                  1
Numero_morts                                  1
Numero_lesionats_lleus                        1
Numero_lesionats_greus                        1
Numero_victimes                               1
Numero_vehicles_implicats                     1
Descripcio_causa_vianant                

In [ ]:
# Convertir las columnas a tipo entero
columns_to_convert = ['Any', 'Mes_any', 'Dia_mes', 'Hora_dia', 'Numero_morts', 'Numero_lesionats_lleus',
                      'Numero_lesionats_greus', 'Numero_victimes', 'Numero_vehicles_implicats',
                      'Codi_carrer5', 'Codi_carrer2' # 'Codi_carrer6', 'Codi_carrer4', 'Codi_carrer3',
                      ]

df_general4[columns_to_convert] = df_general4[columns_to_convert].fillna('-1').astype(int)

In [ ]:
df_general4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62308 entries, 0 to 62307
Data columns (total 39 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Numero_expedient                         62308 non-null  object 
 1   Nom_districte                            62308 non-null  object 
 2   Nom_barri                                62308 non-null  object 
 3   Codi_carrer5                             62308 non-null  int64  
 4   Nom_carrer                               62308 non-null  object 
 5   Descripcio_dia_setmana                   62308 non-null  object 
 6   Any                                      62308 non-null  int64  
 7   Mes_any                                  62308 non-null  int64  
 8   Dia_mes                                  62308 non-null  int64  
 9   Hora_dia                                 62308 non-null  int64  
 10  Descripcio_torn                          62308

### Convertir el *dataset* a .csv

In [ ]:
# Directorio donde se almacenarán los archivos CSV limpios
output_dir = '/content/drive/MyDrive/TFM/clean_data'

# Crear el directorio si no existe
os.makedirs(output_dir, exist_ok=True)

# Guardar el DataFrame general como un archivo CSV
df_general4.to_csv(os.path.join(output_dir, "df.csv"), index=False)

Ahora podemos encontrar el *dataframe* limpio en la ruta '/content/drive/MyDrive/TFM/clean_data'

# Codificación de variables

In [ ]:
os.chdir('/content/drive/MyDrive/TFM/clean_data')

# Leer el archivo CSV
df = pd.read_csv('df.csv')

In [ ]:
df = df.drop(columns=['Nom_barri', 'Codi_carrer6', 'Nom_carrer', 'Longitud', 'Latitud', 'Codi_carrer3',
                      'Descripcio_causa_vianant', 'Codi_carrer2', 'Codi_carrer4', 'Descripcio_model', 'Descripcio_marca',
                      'Descripcio_color', 'Descripcio_situacio', 'Descripcio_victimitzacio', 'Descripcio_Motiu_desplaçament_vianant',
                      'Descripcio_Motiu_desplaçament_conductor', 'Descripcio_Lloc_atropellament_vianant'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62308 entries, 0 to 62307
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Numero_expedient           62308 non-null  object
 1   Nom_districte              62308 non-null  object
 2   Codi_carrer5               62308 non-null  int64 
 3   Descripcio_dia_setmana     62308 non-null  object
 4   Any                        62308 non-null  int64 
 5   Mes_any                    62308 non-null  int64 
 6   Dia_mes                    62308 non-null  int64 
 7   Hora_dia                   62308 non-null  int64 
 8   Descripcio_torn            62308 non-null  object
 9   Descripcio_causa_mediata   62308 non-null  object
 10  Numero_morts               62308 non-null  int64 
 11  Numero_lesionats_lleus     62308 non-null  int64 
 12  Numero_lesionats_greus     62308 non-null  int64 
 13  Numero_victimes            62308 non-null  int64 
 14  Numero

### Nom_districte

In [ ]:
df.Nom_districte.value_counts()

Nom_districte
Eixample               18234
Sant Martí              7733
Sants-Montjuïc          6806
Sarrià-Sant Gervasi     6687
Horta-Guinardó          4517
Les Corts               4261
Sant Andreu             3987
Ciutat Vella            3438
Nou Barris              3411
Gràcia                  3061
Desconegut               173
Name: count, dtype: int64

In [ ]:
df['Eixample']=df['Nom_districte'].str.count('Eixample')
df['Sant_Marti']=df['Nom_districte'].str.count('Sant Martí')
df['Sants_Montjuic']=df['Nom_districte'].str.count('Sants-Montjuïc')
df['Sarria_Sant-Gervasi']=df['Nom_districte'].str.count('Sarrià-Sant Gervasi')
df['Horta_Guinardo']=df['Nom_districte'].str.count('Horta-Guinardó')
df['Les_Corts']=df['Nom_districte'].str.count('Les Corts')
df['Sant_Andreu']=df['Nom_districte'].str.count('Sant Andreu')
df['Ciutat_Vella']=df['Nom_districte'].str.count('Ciutat Vella')
df['Nou_Barris']=df['Nom_districte'].str.count('Nou Barris')
df['Gracia']=df['Nom_districte'].str.count('Gràcia')
df['Districte_Desconegut']=df['Nom_districte'].str.count('Desconegut')

In [ ]:
df['Districte_Desconegut'].unique()

array([0, 1])

### Descripcio_dia_setmana     

In [ ]:
mapeo_dias = {
    'Dilluns': 1,
    'Dimarts': 2,
    'Dimecres': 3,
    'Dijous': 4,
    'Divendres': 5,
    'Dissabte': 6,
    'Diumenge': 7
}

# Mapear los valores de la columna 'Descripcio_dia_setmana' a números usando el diccionario
df['Descripcio_dia_setmana'] = df['Descripcio_dia_setmana'].map(mapeo_dias)

### Descripcio_torn

In [ ]:
mapeo = {
    'Matí': 1,
    'Tarda': 2,
    'Nit': 3
}

# Mapear los valores de la columna 'Descripcio_dia_setmana' a números usando el diccionario
df['Descripcio_torn'] = df['Descripcio_torn'].map(mapeo)

### Descripcio_causa_mediata

In [ ]:
# Definir las categorías y contar sus ocurrencias
df['SinCausaMediata'] = df['Descripcio_causa_mediata'].str.count('No hi ha causa mediata')
df['Alcoholemia'] = df['Descripcio_causa_mediata'].str.count('Alcoholèmia')
df['CalzadaMalEstado'] = df['Descripcio_causa_mediata'].str.count('Calçada en mal estat')
df['VelocidadInadecuada'] = df['Descripcio_causa_mediata'].str.count('Excés de velocitat o inadequada')
df['Drogas'] = df['Descripcio_causa_mediata'].str.count('Drogues o medicament')
df['ObjetosAnimalesEnCalzada'] = df['Descripcio_causa_mediata'].str.count('Objectes o animals a la calçada')
df['FactoresMeteorologicos'] = df['Descripcio_causa_mediata'].str.count('Factors meteorològics')

### Descripcio_tipus_accident

In [ ]:
# Definir las categorías y contar sus ocurrencias
df['TA_Colision'] = df['Descripcio_tipus_accident'].str.count('Col.lisió fronto-lateral')
df['TA_Caida'] = df['Descripcio_tipus_accident'].str.count('Caiguda')
df['TA_Alcance'] = df['Descripcio_tipus_accident'].str.count('Abast')
df['TA_Colision2'] = df['Descripcio_tipus_accident'].str.count('Col.lisió lateral')
df['TA_Atropello'] = df['Descripcio_tipus_accident'].str.count('Atropellament')
df['TA_Colision3'] = df['Descripcio_tipus_accident'].str.count('Col.lisió frontal')
df['TA_Otro'] = df['Descripcio_tipus_accident'].str.count('Altres')
df['TA_ChoqueContraElementoEstático'] = df['Descripcio_tipus_accident'].str.count('Xoc contra element estàtic')
df['TA_CaidaInteriorVehiculo'] = df['Descripcio_tipus_accident'].str.count('Caiguda interior vehicle')
df['TA_Volcada'] = df['Descripcio_tipus_accident'].str.count('Bolcada')
df['TA_Alcance2'] = df['Descripcio_tipus_accident'].str.count('Abast multiple')
df['TA_SalidaDeVia'] = df['Descripcio_tipus_accident'].str.count('Sortida de via amb xoc o col.lisió')
df['TA_Persecución'] = df['Descripcio_tipus_accident'].str.count('Encalç')
df['TA_SalidaDeVia2'] = df['Descripcio_tipus_accident'].str.count('Resta sortides de via')
df['TA_SalidaDeVia3'] = df['Descripcio_tipus_accident'].str.count('Sortida de via amb bolcada')
df['TA_ChoqueContraAnimal'] = df['Descripcio_tipus_accident'].str.count('Xoc amb animal a la calçada')
df['TA_Otro2'] = df['Descripcio_tipus_accident'].str.count('Desconegut')
df['TA_SalidaDeVia4'] = df['Descripcio_tipus_accident'].str.count('Sortida de via amb atropellament')

# Sumar las columnas TA_Colision y TA_Colision2 y almacenar el resultado en la columna TA_Colision
df['TA_Colision'] = df['TA_Colision'] + df['TA_Colision2'] + df['TA_Colision3']
df['TA_SalidaDeVia'] = df['TA_SalidaDeVia2'] + df['TA_SalidaDeVia3'] + df['TA_SalidaDeVia4']
df['TA_Alcance'] = df['TA_Alcance'] + df['TA_Alcance2']
df['TA_Otro'] = df['TA_Otro'] + df['TA_Otro2']

df = df.drop(columns=['TA_Colision2', 'TA_Colision3', 'TA_SalidaDeVia2', 'TA_SalidaDeVia3', 'TA_SalidaDeVia4',
                      'TA_Alcance2', 'TA_Otro2'])

### Descripcio_tipus_vehicle

In [ ]:
df['Descripcio_tipus_vehicle'] = df['Descripcio_tipus_vehicle'].fillna('-1')

In [ ]:
# Definir las categorías y contar sus ocurrencias
df['TV_Taxi'] = df['Descripcio_tipus_vehicle'].str.count('Taxi').fillna('0').astype(int)
df['TV_Motocicleta'] = df['Descripcio_tipus_vehicle'].str.count('Motocicleta|Ciclomotor|Cuadriciclo|Quadricicle').fillna('0').astype(int)
df['TV_Automóvil'] = df['Descripcio_tipus_vehicle'].str.count('Turismo|Turisme|Todo terreno|Autocaravana|Tot terreny').fillna('0').astype(int)
df['TV_Autobús'] = df['Descripcio_tipus_vehicle'].str.count('Autobús|Autobús articulado|Microbus|Microbús|Microbus|Autocar').fillna('0').astype(int)
df['TV_Vehículo_Comercial'] = df['Descripcio_tipus_vehicle'].str.count('Furgoneta|Pick-up').fillna('0').astype(int)
df['TV_Camión'] = df['Descripcio_tipus_vehicle'].str.count('Tractocamión|Camión|Camió rígid|Tractor camió').fillna('0').astype(int)
df['TV_Bicicleta'] = df['Descripcio_tipus_vehicle'].str.count('Bicicleta|Tricicle').fillna('0').astype(int)
df['TV_Vehículo_Personal_Motorizado'] = df['Descripcio_tipus_vehicle'].str.count('Veh. mobilitat personal amb motor').fillna('0').astype(int)

df['TV_Otros'] = df['Descripcio_tipus_vehicle'].str.count('Tranvía|Tren|Tren o tramvia|Otros vehíc. a motor|Maquinaria de obras|\
                                                Maquinŕria d\'obres i serveis|Maquinària d\'obres i serveis|Altres vehicles amb motor|Altres vehicles sense motor|Carro|\
                                                Veh. mobilitat personal sense motor|Ambulància|Ambulŕncia|Desconegut').fillna('0').astype(int)

### Descripcio_carnet

In [ ]:
# Definir las categorías principales y contar sus ocurrencias
df['DC_B'] = df['Descripcio_carnet'].str.count(r'\bB\b').fillna('0').astype(int)
df['DC_Desconegut'] = df['Descripcio_carnet'].str.count('Desconegut|Es desconeix|Sense permís').fillna('0').astype(int)
df['DC_A'] = df['Descripcio_carnet'].str.count(r'\bA\b').fillna('0').astype(int)

# Contar las ocurrencias de las categorías restantes y agruparlas en una columna 'DC_Otros'
df['DC_Otros'] = 0
categorias_otras = ['BTP', 'A1', 'A2', 'AM', 'D', 'C', 'E C', 'D1', 'Llicència', 'E D', 'B1', 'E C1', 'C1', 'E B', 'E D1']
for categoria in categorias_otras:
    df['DC_Otros'] += df['Descripcio_carnet'].str.count(rf'\b{categoria}\b').fillna('0').astype(int)


### Antiguitat_carnet

In [ ]:
df['Antiguitat_carnet'] = df['Antiguitat_carnet'].fillna('-1')

In [ ]:
# Definir los límites de los grupos
bins = [0, 5, 10, 20, 30, 40, 50, 60, 100]
labels = ['AC_0-5', 'AC_5-10', 'AC_10-20',
          'AC_20-30', 'AC_30-40', 'AC_40-50',
          'AC_50-60', 'AC_más de 60']

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Inicializar contadores para cada grupo de antigüedad de carnet
    contador_antiguedad = {label: 0 for label in labels}
    # Obtener las antigüedades de carnet de la observación y dividirlas
    antiguedades = [int(x.strip()) for x in row['Antiguitat_carnet'].split(',') if x.strip().isdigit()]
    # Contar las ocurrencias de cada grupo de antigüedad de carnet para estas antigüedades
    for antiguedad in antiguedades:
        for i in range(len(bins) - 1):
            if bins[i] <= antiguedad < bins[i + 1]:
                contador_antiguedad[labels[i]] += 1
                break
    # Agregar las cuentas como nuevas columnas en el DataFrame
    for label, count in contador_antiguedad.items():
        df.at[index, label] = count

# Convertir las nuevas columnas a 'int64'
df[labels] = df[labels].astype('int64')

### Descripcio_sexe

In [ ]:
# Definir las categorías y contar sus ocurrencias
df['Hombre'] = df['Descripcio_sexe'].str.count('Home')
df['Mujer'] = df['Descripcio_sexe'].str.count('Dona')
df['Desconocido'] = df['Descripcio_sexe'].str.count('Desconegut')

### Edat

In [ ]:
df['Edat'] = df['Edat'].fillna('-1')

In [ ]:
# Definir los límites de los grupos
bins = [0, 17, 25, 35, 45, 60, float('inf')]
labels = ['Edad_1-17', 'Edad_18-25', 'Edad_26-35', 'Edad_36-45', 'Edad_46-60', 'Edad_más de 60']

# Iterar sobre las filas del DataFrame
for index, row in df.iterrows():
    # Inicializar contadores para cada grupo de edad
    contador_edades = {label: 0 for label in labels}
    # Obtener las edades de la observación y dividirlas
    edades = [int(x.strip()) for x in row['Edat'].split(',') if x.strip().isdigit()]
    # Contar las ocurrencias de cada grupo de edad para estas edades
    for edad in edades:
        for i in range(len(bins) - 1):
            if bins[i] <= edad < bins[i + 1]:
                contador_edades[labels[i]] += 1
                break
    # Agregar las cuentas como nuevas columnas en el DataFrame
    for label, count in contador_edades.items():
        df.at[index, label] = count

# Convertir las nuevas columnas a 'int64'
df[labels] = df[labels].astype('int64')

### Descripcio_tipus_persona

In [ ]:
# Definir las categorías y contar sus ocurrencias
df['TP_Conductor'] = df['Descripcio_tipus_persona'].str.count('Conductor')
df['TP_Pasajero'] = df['Descripcio_tipus_persona'].str.count('Passatger')
df['TP_Peaton'] = df['Descripcio_tipus_persona'].str.count('Vianant')
df['TP_Desconocido'] = df['Descripcio_tipus_persona'].str.count('Desconegut')

In [ ]:
df = df.drop(columns=['Nom_districte','Descripcio_sexe', 'Descripcio_causa_mediata', 'Descripcio_tipus_accident', 'Descripcio_tipus_vehicle', 'Descripcio_carnet', 'Antiguitat_carnet', 'Edat', 'Descripcio_tipus_persona'])

In [ ]:
df['Hombre'] = df['Hombre'].fillna('0').astype(int)
df['Mujer'] = df['Mujer'].fillna('0').astype(int)
df['Desconocido'] = df['Desconocido'].fillna('1').astype(int)
df['TP_Conductor'] = df['TP_Conductor'].fillna('0').astype(int)
df['TP_Pasajero'] = df['TP_Pasajero'].fillna('0').astype(int)
df['TP_Peaton'] = df['TP_Peaton'].fillna('0').astype(int)
df['TP_Desconocido'] = df['TP_Desconocido'].fillna('1').astype(int)

In [ ]:
df.isna().sum()

Numero_expedient                   0
Codi_carrer5                       0
Descripcio_dia_setmana             0
Any                                0
Mes_any                            0
Dia_mes                            0
Hora_dia                           0
Descripcio_torn                    0
Numero_morts                       0
Numero_lesionats_lleus             0
Numero_lesionats_greus             0
Numero_victimes                    0
Numero_vehicles_implicats          0
Eixample                           0
Sant_Marti                         0
Sants_Montjuic                     0
Sarria_Sant-Gervasi                0
Horta_Guinardo                     0
Les_Corts                          0
Sant_Andreu                        0
Ciutat_Vella                       0
Nou_Barris                         0
Gracia                             0
Districte_Desconegut               0
SinCausaMediata                    0
Alcoholemia                        0
CalzadaMalEstado                   0
V

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62308 entries, 0 to 62307
Data columns (total 76 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Numero_expedient                 62308 non-null  object
 1   Codi_carrer5                     62308 non-null  int64 
 2   Descripcio_dia_setmana           62308 non-null  int64 
 3   Any                              62308 non-null  int64 
 4   Mes_any                          62308 non-null  int64 
 5   Dia_mes                          62308 non-null  int64 
 6   Hora_dia                         62308 non-null  int64 
 7   Descripcio_torn                  62308 non-null  int64 
 8   Numero_morts                     62308 non-null  int64 
 9   Numero_lesionats_lleus           62308 non-null  int64 
 10  Numero_lesionats_greus           62308 non-null  int64 
 11  Numero_victimes                  62308 non-null  int64 
 12  Numero_vehicles_implicats       

In [ ]:
df.head()

,Numero_expedient,Codi_carrer5,Descripcio_dia_setmana,Any,Mes_any,Dia_mes,Hora_dia,Descripcio_torn,Numero_morts,Numero_lesionats_lleus,Numero_lesionats_greus,Numero_victimes,Numero_vehicles_implicats,Eixample,Sant_Marti,Sants_Montjuic,Sarria_Sant-Gervasi,Horta_Guinardo,Les_Corts,Sant_Andreu,Ciutat_Vella,Nou_Barris,Gracia,Districte_Desconegut,SinCausaMediata,Alcoholemia,CalzadaMalEstado,VelocidadInadecuada,Drogas,ObjetosAnimalesEnCalzada,FactoresMeteorologicos,TA_Colision,TA_Caida,TA_Alcance,TA_Atropello,TA_Otro,TA_ChoqueContraElementoEstático,TA_CaidaInteriorVehiculo,TA_Volcada,TA_SalidaDeVia,TA_Persecución,TA_ChoqueContraAnimal,TV_Taxi,TV_Motocicleta,TV_Automóvil,TV_Autobús,TV_Vehículo_Comercial,TV_Camión,TV_Bicicleta,TV_Vehículo_Personal_Motorizado,TV_Otros,DC_B,DC_Desconegut,DC_A,DC_Otros,AC_0-5,AC_5-10,AC_10-20,AC_20-30,AC_30-40,AC_40-50,AC_50-60,AC_más de 60,Hombre,Mujer,Desconocido,Edad_1-17,Edad_18-25,Edad_26-35,Edad_36-45,Edad_46-60,Edad_más de 60,TP_Conductor,TP_Pasajero,TP_Peaton,TP_Desconocido
0,2016S000001,144601,5,2016,1,1,0,3,0,1,0,1,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1,2016S000002,194202,5,2016,1,1,2,3,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0
2,2016S000003,209900,5,2016,1,1,5,3,0,1,0,1,2,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
3,2016S000004,32700,5,2016,1,1,6,1,0,3,0,3,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,0,0,0,0,1,1,0,0,0,0,0,1,2,0,0,0,1,1,1,0,1,1,1,0
4,2016S000005,269902,5,2016,1,1,8,1,0,1,0,1,2,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0


In [193]:
# Directorio donde se almacenarán los archivos CSV limpios
output_dir = '/content/drive/MyDrive/TFM/clean_data'

# Crear el directorio si no existe
os.makedirs(output_dir, exist_ok=True)

# Guardar el DataFrame general como un archivo CSV
df.to_csv(os.path.join(output_dir, "df_encoded.csv"), index=False)


Ahora podemos encontrar el *dataframe* limpio en la ruta '/content/drive/MyDrive/TFM/clean_data'

## Comprobaciones

### Inconsistencias tipo de vehículo: df_personas y df_tiposdevehiculo

In [ ]:
df6_veh = df_6.groupby('Numero_expedient')['Desc_Tipus_vehicle_implicat'].value_counts()

In [ ]:
df4_veh = df_4.groupby('Numero_expedient')['Descripcio_tipus_vehicle'].value_counts()

In [ ]:
df4_veh.reset_index()

,Numero_expedient,Descripcio_tipus_vehicle,count
0,2016S000001,Taxi,1
1,2016S000001,Turismo,1
2,2016S000002,Motocicleta,1
3,2016S000003,Turismo,1
4,2016S000003,Motocicleta,1
...,...,...,...
103510,2022S008013,Taxi,1
103511,2022S008013,Motocicleta,1
103512,2023S000004,Motocicleta,1
103513,2023S000004,Bicicleta,1


In [ ]:
df6_veh.reset_index()

,Numero_expedient,Desc_Tipus_vehicle_implicat,count
0,2016S000001,Taxi,1
1,2016S000002,Motocicleta,1
2,2016S000003,Motocicleta,1
3,2016S000004,Turismo,3
4,2016S000005,Turismo,1
...,...,...,...
64084,2022S008011,Turisme,1
64085,2022S008012,Motocicleta,1
64086,2022S008013,Motocicleta,1
64087,2023S000004,Bicicleta,1


In [ ]:
df4_veh.reset_index().query('Numero_expedient	== "2016S000004"')

,Numero_expedient,Descripcio_tipus_vehicle,count
5,2016S000004,Turismo,1
6,2016S000004,Taxi,1


In [ ]:
df6_veh.reset_index().query('Numero_expedient	== "2016S000004"')

,Numero_expedient,Desc_Tipus_vehicle_implicat,count
3,2016S000004,Turismo,3


In [ ]:
df4_veh.reset_index().query('Numero_expedient	== "2016S000001"')

,Numero_expedient,Descripcio_tipus_vehicle,count
0,2016S000001,Taxi,1
1,2016S000001,Turismo,1


In [ ]:
df6_veh.reset_index().query('Numero_expedient	== "2016S000001"')

,Numero_expedient,Desc_Tipus_vehicle_implicat,count
0,2016S000001,Taxi,1


In [ ]:
df4_veh.reset_index().query('count	>= 5').head(40)

,Numero_expedient,Descripcio_tipus_vehicle,count
246,2016S000152,Turismo,5
615,2016S000371,Turismo,5
1235,2016S000745,Motocicleta,5
2224,2016S001337,Turismo,5
3600,2016S002171,Turismo,5
3674,2016S002219,Turismo,5
3992,2016S002411,Turismo,6
4015,2016S002424,Turismo,5
4050,2016S002448,Turismo,6
4166,2016S002515,Turismo,5


In [ ]:
df4_veh.reset_index().query('Numero_expedient	== "2016S006461"')

,Numero_expedient,Descripcio_tipus_vehicle,count
10765,2016S006461,Motocicleta,8
10766,2016S006461,Turismo,1


In [ ]:
df6_veh.reset_index().query('Numero_expedient	== "2016S006461"')

,Numero_expedient,Desc_Tipus_vehicle_implicat,count
